In [1]:
# Import dictionary
import pandas as pd
import pypyodbc as py
import xlsxwriter
import gspread
import gspread_dataframe as gd
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials # Đọc dữ liệu từ googlesheets
from df2gspread import df2gspread as d2g # Ghi dữ liệu lên googlesheets
from pprint import pprint
from googleapiclient import discovery
import pypyodbc #connect với sql server
import numpy as np #Tính toán 😊,-,*,/)
import sqlalchemy #query trực tiếp
import urllib #connect url
import time #
import datetime as dt
from datetime import timedelta, datetime
import pymssql
import calendar
from google.oauth2 import service_account
import pygsheets

In [2]:
# Connect to DB
db = py.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=103.69.193.246;"
                        "Database=dwh;"
                
                        "uid=dwh_qtrr;pwd=DWH@qtrr")
print('Connected to SQL server: DWH')

Connected to SQL server: DWH


In [3]:
# Thư viện tương tác với googlesheets
import pygsheets
import json
from google.oauth2 import service_account



# open key to get data from GGsheets
with open(r'D:\F88\Python code\Immediate_warning\service_account.json') as source:
    info = json.load(source)
credentials = service_account.Credentials.from_service_account_info(info)

client = pygsheets.authorize(service_account_file=r'D:\F88\Python code\Immediate_warning\service_account.json')

# collecting shop information from business department
shop_id='1OCzHxTE7Er8_W1-IT92rTOCISAlufFOjcwtw2eIX0tU'
shop_wb=client.open_by_key(shop_id)
shop_data=shop_wb.worksheet_by_title('Everything').get_as_df()

# another way to get data from GGsheet
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(r'D:\F88\Python code\Immediate_warning\service_account.json', scope)
gc = gspread.authorize(credentials)

#Function chuyển sheet thành Data Frame
def to_dataframe(wb_name,sheet_name,data_row,col):
    a=wb_name.worksheet(sheet_name).get_all_values()
    return pd.DataFrame(data=a[data_row:],columns=a[col])

#Funtion update data vào sheet wb_name:tên wb,target_sheet: tên sheet cần update; data:dữ liệu muốn update
def update_data(wb_name,target_sheet,data):
    existing =to_dataframe(wb_name,target_sheet,1,0)
    updated = existing.append(data)
    gd.set_with_dataframe(wb_name.worksheet(target_sheet),updated)

# Function ghi đè dữ liệu lên gg sheet
def write_data(wb_name,ws_name,ws_range,data):
    wb_name.values_clear(ws_range)
    sheet=wb_name.worksheet(ws_name)
    set_with_dataframe(sheet,data)
    print('Đã ghi dữ liệu lên sheet '+ ws_name)

# get shop information from marketing
# Lấy ID shopdetail
shop_detail_id='1ZTQE_pfBCAUr-0GSMPJBqtCyZVxX4hdiPuT_RDrdjeQ'
shop_detail_wb=gc.open_by_key(shop_detail_id)
shop_data_sub=to_dataframe(shop_detail_wb,'shop_detail',1,0)

In [4]:
# shop_data.to_excel('shop_data.xlsx', index = False)

In [5]:
#  Check date = today() - 1
check_date= (dt.date.today()+dt.timedelta(days=-1)).strftime('%Y-%m-%d')
# shop active on Pos
shop_detail_pos = pd.read_sql_query("""Declare @check_date date = '"""+check_date+"""'
select  sd2.code, sd2.name
from (select distinct(sd.shopcode) shop_code
from (SELECT case when code = 'SGN0120' then 'SGN0147'
when code = 'SG2046' then 'SGN0126'
when code = 'SGN0107' then 'SG2014'
else code end shopcode
  FROM [dwh].[dbo].[ShopDetail]
  where code not in ('TEST','HS', 'F88LUX','HNLTK','HNIXLN01','SGNXLN01','HNBM1','HOISO')) sd
  left join (select shopcode, max(fromdate) max_cre
	from pawn group by shopcode) pa on pa.shopcode = sd.shopcode
	where pa.max_cre >= dateadd(d, -15, @check_date)) main
left join [dwh].[dbo].[ShopDetail] sd2 on main.shop_code = sd2.code""", db)

# select field from shop infor of business department
shop_data = shop_data[['Tên PGD', 'Mã PGD', 'Phường/Xã', 'Quận/Huyện',
       'Tỉnh/TP', 'Vĩ độ', 'Kinh độ','Shop ID','Vùng/Miền','Thời gian KT', 'Vùng']]
shop_data.rename(columns={'Tỉnh/TP':'Tỉnh/ Thành phố'}, inplace=True)

# Adding conditions to shop information
shop_data = shop_data.merge(shop_detail_pos, left_on='Mã PGD', right_on='code', how = 'left')
shop_data = shop_data[shop_data['name'].isnull() == False]

shop_data_sub = shop_data_sub[['Mã PGD', 'GĐM', 'TPK', 'QLKV']]

shop_data = shop_data.merge(shop_data_sub, on = 'Mã PGD', how = 'left')

# Get writeoff list
writeoff_id = '15r30_u_xE0iUexkUgsbZqcZTy0htmSflCp-uty_1hWA'
writeoff_wb = client.open_by_key(writeoff_id)
write_off_list=writeoff_wb.worksheet_by_title('list').get_as_df()

write_off_list = write_off_list[['write_off_period', 'codeno', 'amount']]
write_off_list.columns = ['write_off_period', 'pawn_code', 'amount']

ONTIME RATE

In [6]:
# Car Title
def ontime_dkot(fromdate, todate, n_days_after):
    monthly_ontime_str = """Declare @schedule_from date ='"""+fromdate+"""'
Declare @schedule_to date ='"""+todate+"""'
select 'monthly_org_paid' title_type, *,case when (pmt_stt like N'overdue' and dpd_"""+n_days_after+"""_day_after is not null and dpd_"""+n_days_after+"""_day_after>0) then N'still_over_due'
when pmt_stt like N'overdue' and (dpd_"""+n_days_after+"""_day_after is null or dpd_"""+n_days_after+"""_day_after=0) and t_"""+n_days_after+"""_day_after<getdate() then N'paid_after_"""+n_days_after+"""_day'
when pmt_stt like N'overdue' and t_"""+n_days_after+"""_day_after>=getdate() then N'out_of_range'
else N'ontime' end as stt_after_"""+n_days_after+"""_day,
case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' 
else N'Vay thường' end as package_type
 from (
select  pd.pawnid,po.tuoino dpd_yesterday,po2.TuoiNo dpd_"""+n_days_after+"""_day_after,pa.PackageCode,pa.shopname, pa.shopcode,areaid,
pd.scheduletodate,
dateadd(day,"""+n_days_after+""",pd.ScheduleToDate) t_"""+n_days_after+"""_day_after,pd.acc_sum,pd.paydate,
pd.acc_prin_payment,paidmoney,pa.status,pa.CodeNo,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt, pa.cus_type,  
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=(acc_sum-1000)) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt,
case when pa.frequency <= 30 then N'1. Kỳ hạn 1 tháng'
  when pa.frequency <= 90 then N'2. Kỳ hạn 3 tháng'
  when pa.frequency <= 180 then N'3. Kỳ hạn 6 tháng'
  when pa.frequency <= 270 then N'4. Kỳ hạn 9 tháng'
  when pa.frequency <= 360 then N'5. Kỳ hạn 12 tháng'
  else N'6. >12 tháng' end package_time_range,
  case when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 50000000 then '0. <=50tr'
when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 100000000 then '2. 100-200tr'
when pa.PackageCode is not null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) > 200000000 then N'4. >200tr'
when pa.PackageCode is null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 300000000 then '3. 200-300tr'
else N'4. >300tr' end "package_money_range",(pa.LoanMoneyOrg - pa.[InsurranceMoney]) net_disbur, loanmoneyorg
from pawnoverduedetail pd  
left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))
left join pawnoverdue po2 on (po2.pawnid=pd.pawnid and po2.created=dateadd(day,"""+n_days_after+""",pd.scheduletodate))
left join  (select pa1.*,shop.name shopname,shop.areaid,
case when Lag (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) is null then N'Khách mới' else N'Khách cũ' end "cus_type" from Pawn pa1 
left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid 
left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid  
 where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt)) 
 and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%15%'
  and InterestMoney like N'%goc%' and pd.scheduletodate<@schedule_to and pd.scheduletodate>=@schedule_from) tab
  where dpd_yesterday<=10
  order by pawnid,scheduletodate"""
    eop_ontime_str= """Declare @schedule_from date ='"""+fromdate+"""'
Declare @schedule_to date ='"""+todate+"""'
select 'eop_org_paid' title_type, *,case when (pmt_stt like N'overdue' and dpd_"""+n_days_after+"""_day_after is not null and dpd_"""+n_days_after+"""_day_after>0) then N'still_over_due'
when pmt_stt like N'overdue' and (dpd_"""+n_days_after+"""_day_after is null or dpd_"""+n_days_after+"""_day_after=0) and t_"""+n_days_after+"""_day_after<getdate() then N'paid_after_"""+n_days_after+"""_day'
when pmt_stt like N'overdue' and t_"""+n_days_after+"""_day_after>=getdate() then N'out_of_range'
else N'ontime' end as stt_after_"""+n_days_after+"""_day,
case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
 from (
select  ROW_NUMBER() over (partition by pd.pawnid order by moneyinterest) as row_num,pd.pawnid,po.tuoino dpd_yesterday,
po2.TuoiNo dpd_"""+n_days_after+"""_day_after,pa.PackageCode,pa.shopname,pa.shopcode,areaid,pd.scheduletodate,dateadd(day,"""+n_days_after+""",pd.ScheduleToDate) t_"""+n_days_after+"""_day_after
,pd.acc_sum,pd.paydate,
pd.acc_prin_payment,paidmoney,pa.status,pa.CodeNo,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt, pa.cus_type,  
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=(acc_sum-1000)) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt,
case when pa.frequency <= 30 then N'1. Kỳ hạn 1 tháng'
  when pa.frequency <= 90 then N'2. Kỳ hạn 3 tháng'
  when pa.frequency <= 180 then N'3. Kỳ hạn 6 tháng'
  when pa.frequency <= 270 then N'4. Kỳ hạn 9 tháng'
  when pa.frequency <= 360 then N'5. Kỳ hạn 12 tháng'
  else N'6. >12 tháng' end package_time_range,
  case when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 50000000 then '0. <=50tr'
when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 100000000 then '2. 100-200tr'
when pa.PackageCode is not null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) > 200000000 then N'4. >200tr'
when pa.PackageCode is null and  (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 300000000 then '3. 200-300tr'
else N'4. >300tr' end "package_money_range", (pa.LoanMoneyOrg - pa.[InsurranceMoney]) net_disbur, loanmoneyorg
from pawncaroverdue pd  
left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))
left join pawnoverdue po2 on (po2.pawnid=pd.pawnid and po2.created=dateadd(day,"""+n_days_after+""",pd.scheduletodate))
left join  (select pa1.*,shop.name shopname,shop.areaid,
case when Lag (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) is null then N'Khách mới' else N'Khách cũ' end "cus_type"
from Pawn pa1 
left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid 
left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid  
 where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt)) 
 and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%15%'
  and pd.scheduletodate<=@schedule_to and pd.scheduletodate>=@schedule_from) tab
  where dpd_yesterday<=10 and row_num=1
  order by pawnid,scheduletodate"""
    monthly_ontime = pd.read_sql_query(monthly_ontime_str,db)
    eop_ontime = pd.read_sql_query(eop_ontime_str,db)
    ontime_data = pd.concat([monthly_ontime, eop_ontime], ignore_index=True)
    ontime_data = ontime_data.merge(write_off_list, left_on = 'codeno', right_on = 'pawn_code',how = 'left')
    ontime_data = ontime_data[ontime_data['pawn_code'].isnull()==True]
    column_name='stt_after_'+n_days_after+'_day'
    colmn_name_2='paid_after_'+n_days_after+'_day'
    ontime_data=pd.merge(ontime_data,shop_data, left_on='shopcode',right_on='Mã PGD',how='left')
    ontime_data = ontime_data[ontime_data['Vùng/Miền'].isnull() == False]
    ontime_data['khu_vuc'] = np.where(ontime_data['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(ontime_data['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((ontime_data['Tỉnh/ Thành phố'] != 'Hà Nội') & (ontime_data['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((ontime_data['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (ontime_data['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    ontime_data.insert(0,'categoryname','Đăng ký Ô tô')
    ontime_data['brr_ym'] = [i.strftime("%Y%m") for i in ontime_data['fromdate']]
    return ontime_data

In [7]:
#  Ontime Motor Title
def ontime_dkxm(fromdate,todate,n_days_after_num):
    n_days_after=n_days_after_num
    ontime_dkxm_str = """Declare @schedule_from date ='"""+fromdate+"""'
Declare @schedule_to date ='"""+todate+"""'
select 'motor_title' title_type,*,case when (pmt_stt like N'overdue' and dpd_"""+n_days_after+"""_day_after is not null and dpd_"""+n_days_after+"""_day_after>0) then N'still_over_due'
when pmt_stt like N'overdue' and (dpd_"""+n_days_after+"""_day_after is null or dpd_"""+n_days_after+"""_day_after=0) and t_"""+n_days_after+"""_day_after<getdate() then N'paid_after_"""+n_days_after+"""_day'
when pmt_stt like N'overdue' and t_"""+n_days_after+"""_day_after>=getdate() then N'out_of_range'
else N'ontime' end as stt_after_"""+n_days_after+"""_day,case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
 from (
select  pd.pawnid,po.tuoino dpd_yesterday,po2.TuoiNo dpd_"""+n_days_after+"""_day_after,pa.PackageCode, pa.shopname,pa.shopcode,
areaid,pd.scheduletodate,
dateadd(day,"""+n_days_after_num+""",pd.ScheduleToDate) t_"""+n_days_after+"""_day_after,pd.principalmoney,pd.acc_sum,pd.paydate,
pd.acc_prin_payment,paidmoney,pa.status,pa.CodeNo,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,  pa.cus_type,
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=(acc_sum-1000)) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt,
case when pa.frequency <= 30 then N'1. Kỳ hạn 1 tháng'
  when pa.frequency <= 90 then N'2. Kỳ hạn 3 tháng'
  when pa.frequency <= 180 then N'3. Kỳ hạn 6 tháng'
  when pa.frequency <= 270 then N'4. Kỳ hạn 9 tháng'
  when pa.frequency <= 360 then N'5. Kỳ hạn 12 tháng'
  else N'6. >12 tháng' end package_time_range,
  case when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 3000000 then '0. <=3tr'
		when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 10000000 then '1. 3-10tr'
		when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 15000000 then '2. 10-15tr'
		when (pa.LoanMoneyOrg - pa.[InsurranceMoney]) <= 25000000 then '3. 15-25tr'
		else '4. >25tr' end "package_money_range", (pa.LoanMoneyOrg - pa.[InsurranceMoney]) net_disbur, loanmoneyorg
from pawnoverduedetail pd  
left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))
left join pawnoverdue po2 on (po2.pawnid=pd.pawnid and po2.created=dateadd(day,"""+n_days_after_num+""",pd.scheduletodate))
left join  (select pa1.*,shop.name shopname,shop.areaid,
case when Lag (pa1.Pawnid) over (PARTITION BY pa1.customerid, 
pa1.[CategoryCode] ORDER BY pa1.fromdate asc) is null then N'Khách mới' else N'Khách cũ' end "cus_type" 
from Pawn pa1 
left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid 
left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid  
 where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt)) 
 and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%17%'
  and InterestMoney like N'%goc%' and pd.scheduletodate<=@schedule_to and pd.scheduletodate>=@schedule_from) tab
  where dpd_yesterday<=10
  order by pawnid,scheduletodate"""
#     return ontime_dkxm_str
    ontime_data=pd.read_sql_query(ontime_dkxm_str,db)
    ontime_data = ontime_data.merge(write_off_list, left_on = 'codeno', right_on = 'pawn_code',how = 'left')
    ontime_data = ontime_data[ontime_data['pawn_code'].isnull()==True]
    column_name='stt_after_'+n_days_after+'_day'
    colmn_name_2='paid_after_'+n_days_after+'_day'
    ontime_data=pd.merge(ontime_data,shop_data,left_on='shopcode',right_on='Mã PGD',how='left')
    ontime_data = ontime_data[ontime_data['Vùng/Miền'].isnull() == False]
    ontime_data['khu_vuc'] = np.where(ontime_data['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(ontime_data['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((ontime_data['Tỉnh/ Thành phố'] != 'Hà Nội') & (ontime_data['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((ontime_data['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (ontime_data['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN','Miền Trung'))))
    ontime_data.insert(0,'categoryname','Đăng ký xe máy')
    ontime_data['brr_ym'] = [i.strftime("%Y%m") for i in ontime_data['fromdate']]
    #     ontime_pivot=pd.pivot_table(ontime_data,ikto máy')=['areaid','shopname', 'Miền', 'Tỉnh/ Thành phố','package_type'],columns=column_name,values='pawnid',aggfunc='count',margins=True).reset_index()
#     ontime_pivot.insert(0,'categoryname','Đăng ký xe máy')
#     ontime_pivot['ontime_rate']=ontime_pivot.ontime/ontime_pivot.All
#     ontime_pivot['paid_when_dpd_less_than_'+n_days_after_num+'_rate']=(ontime_pivot.ontime+ontime_pivot[colmn_name_2])/ontime_pivot.All
#     ontime_pivot.to_excel(r'D:\F88\Python code\ontime_dkot\ontime_dkxm_'+fromdate+"_"+todate+'.xlsx', index=False)
    
#     ontime_data['todate_ym'] = datetime.strptime(ontime['todate'], '%Y-%m-%d').strftime('%Y%m')
#     ontime_data['fromdate_ym'] = datetime.strptime(ontime['fromdate'], '%Y-%m-%d').strftime('%Y%m')
    return ontime_data

In [8]:
#  Starting day in time range
n_days_after = '3'
cb_day= dt.date.today()+dt.timedelta(days=-1)

# Time range
# compare time range
compare_range=6
date_list = []
ob_list = []
cb_list=[]
for i in range (1,compare_range):
    last_day=calendar.monthrange(cb_day.year,cb_day.month-i)[1]
    last_day_2=calendar.monthrange(cb_day.year,cb_day.month-i+1)[1]
    ob_day = dt.datetime(cb_day.year,cb_day.month-i,last_day)+ dt.timedelta(days = 1)
    ob_day = ob_day.strftime("%Y-%m-%d")
    cb_day_2=dt.datetime(cb_day.year,cb_day.month-i+1,min(last_day_2,cb_day.day)).strftime("%Y-%m-%d")
    ob_list.append(ob_day)
    cb_list.append(cb_day_2)                     
ext_date_dict = dict(zip(ob_list, cb_list))
for i in ext_date_dict.items():
    i = i + (n_days_after,)
    date_list.append(i)

In [9]:
date_list

[('2020-12-01', '2020-12-28', '3'),
 ('2020-11-01', '2020-11-28', '3'),
 ('2020-10-01', '2020-10-28', '3'),
 ('2020-09-01', '2020-09-28', '3'),
 ('2020-08-01', '2020-08-28', '3')]

In [10]:
# raw data thiss month (m)
ontime_dkot_this_month = ontime_dkot(date_list[0][0],date_list[0][1],n_days_after)
ontime_dkxm_this_month = ontime_dkxm(date_list[0][0],date_list[0][1],n_days_after)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [11]:
# raw data last month (m-1)
ontime_dkot_last_month = ontime_dkot(date_list[1][0],date_list[1][1],n_days_after)
ontime_dkxm_last_month = ontime_dkxm(date_list[1][0],date_list[1][1],n_days_after)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [12]:
# Bảng tổng hợp data raw của DKXM
ontime_spdk = ontime_dkot_this_month.append( ontime_dkxm_this_month, ignore_index = True)
ontime_spdk = ontime_spdk.fillna(0)
ontime_spdk_last_month = ontime_dkot_last_month.append( ontime_dkxm_last_month, ignore_index = True)
ontime_spdk_last_month = ontime_spdk_last_month.fillna(0)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [13]:
#  categoryname, package_type, area
ontime_overview_1 = ontime_spdk.pivot_table(columns=['categoryname', 'package_type','Vùng/Miền'], index ='pmt_stt', values ='pawnid',aggfunc ='nunique').reset_index()
ontime_overview_2 = ontime_spdk.pivot_table(columns=['categoryname', 'package_type','Vùng/Miền'], index ='stt_after_3_day', values ='pawnid', aggfunc ='nunique').reset_index()
ontime_overview_1_last_month = ontime_spdk_last_month.pivot_table(columns=['categoryname', 'package_type','Vùng/Miền'], index ='pmt_stt', values ='pawnid',aggfunc ='nunique').reset_index()
ontime_overview_2_last_month = ontime_spdk_last_month.pivot_table(columns=['categoryname', 'package_type','Vùng/Miền'], index ='stt_after_3_day', values ='pawnid', aggfunc ='nunique').reset_index()

In [14]:
ontime_overview_1 = ontime_overview_1.fillna(0)
ontime_overview_2 = ontime_overview_2.fillna(0)
ontime_overview_1_last_month = ontime_overview_1_last_month.fillna(0)
ontime_overview_2_last_month = ontime_overview_2_last_month.fillna(0)

In [15]:
# category, area
ontime_khuvuc_1 = ontime_spdk.pivot_table(columns='khu_vuc', index = ['categoryname','pmt_stt'], values='pawnid', aggfunc='nunique').reset_index()
ontime_khuvuc_2 = ontime_spdk.pivot_table(columns='khu_vuc', index = ['categoryname','stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()
ontime_khuvuc_1_last_month = ontime_spdk_last_month.pivot_table(columns='khu_vuc', index = ['categoryname','pmt_stt'], values='pawnid', aggfunc='nunique').reset_index()
ontime_khuvuc_2_last_month = ontime_spdk_last_month.pivot_table(columns='khu_vuc', index = ['categoryname','stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()

In [16]:
ontime_khuvuc_1 = ontime_khuvuc_1[['categoryname','pmt_stt','Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_2 = ontime_khuvuc_2[['categoryname','stt_after_3_day','Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_1_last_month = ontime_khuvuc_1_last_month[['categoryname','pmt_stt','Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_2_last_month = ontime_khuvuc_2_last_month[['categoryname','stt_after_3_day','Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_1 = ontime_khuvuc_1.fillna(0)
ontime_khuvuc_2 = ontime_khuvuc_2.fillna(0)
ontime_khuvuc_1_last_month = ontime_khuvuc_1_last_month.fillna(0)
ontime_khuvuc_2_last_month = ontime_khuvuc_2_last_month.fillna(0)

In [17]:
# area, category, package
ontime_khuvuc_package_1 = ontime_spdk.pivot_table(columns='khu_vuc', index = ['categoryname', 'package_type','pmt_stt'], values='pawnid', aggfunc='nunique').reset_index()
ontime_khuvuc_package_2 = ontime_spdk.pivot_table(columns='khu_vuc', index = ['categoryname', 'package_type','stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()
ontime_khuvuc_package_1_last_month = ontime_spdk_last_month.pivot_table(columns='khu_vuc', index = ['categoryname', 'package_type','pmt_stt'], values='pawnid', aggfunc='nunique').reset_index()
ontime_khuvuc_package_2_last_month = ontime_spdk_last_month.pivot_table(columns='khu_vuc', index = ['categoryname', 'package_type','stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()

In [18]:
ontime_khuvuc_package_1 = ontime_khuvuc_package_1[['categoryname', 'package_type', 'pmt_stt', 'Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_package_2 = ontime_khuvuc_package_2[['categoryname', 'package_type', 'stt_after_3_day', 'Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_package_1_last_month = ontime_khuvuc_package_1_last_month[['categoryname', 'package_type', 'pmt_stt', 'Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
ontime_khuvuc_package_2_last_month = ontime_khuvuc_package_2_last_month[['categoryname', 'package_type', 'stt_after_3_day', 'Hà Nội','Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]

In [19]:
# shop, category
ontime_pgd = ontime_spdk.pivot_table(index=['categoryname','khu_vuc','shopname'], columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_pgd_last_month = ontime_spdk_last_month.pivot_table(index=['categoryname','khu_vuc','shopname'], columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_pgd = ontime_pgd.fillna(0)
ontime_pgd_last_month = ontime_pgd_last_month.fillna(0)

In [20]:
# ontime_pgd['All']=ontime_pgd['ontime']+ontime_pgd['paid_after_3_day']
# ontime_pgd['ontime_rate'] = ontime_pgd['ontime']/ontime_pgd['All']
# ontime_pgd['pmt_after_3days'] = (ontime_pgd['ontime']+ontime_pgd['paid_after_3_day'])/ontime_pgd['All']
# ontime_pgd = ontime_pgd.sort_values(by=['ontime_rate'])
# ontime_pgd_last_month['All']=ontime_pgd_last_month['ontime']+ontime_pgd_last_month['paid_after_3_day']+ontime_pgd_last_month['still_over_due']
# ontime_pgd_last_month['ontime_rate'] = ontime_pgd_last_month['ontime']/ontime_pgd_last_month['All']
# ontime_pgd_last_month['pmt_after_3days'] = (ontime_pgd_last_month['ontime']+ontime_pgd_last_month['paid_after_3_day'])/ontime_pgd_last_month['All']
# ontime_pgd_last_month = ontime_pgd_last_month.sort_values(by=['ontime_rate'])

In [21]:
ontime_pgd.head(5)

stt_after_3_day,categoryname,khu_vuc,shopname,ontime,out_of_range,paid_after_3_day,still_over_due
0,Đăng ký xe máy,Hà Nội,F88 - 505 Ngọc Hồi,97.0,3.0,23.0,22.0
1,Đăng ký xe máy,Hà Nội,F88 - 59 Ngô Gia Tự,270.0,4.0,56.0,50.0
2,Đăng ký xe máy,Hà Nội,F88 01 Cầu Tó,185.0,4.0,41.0,34.0
3,Đăng ký xe máy,Hà Nội,F88 01 Đại Từ,153.0,3.0,28.0,12.0
4,Đăng ký xe máy,Hà Nội,F88 01 Đội Cấn,153.0,2.0,30.0,7.0


In [22]:
ontime_pgd['All']=ontime_pgd['ontime']+ontime_pgd['paid_after_3_day']+ontime_pgd['still_over_due']
ontime_pgd['ontime_rate'] = ontime_pgd['ontime']/ontime_pgd['All']
ontime_pgd['pmt_after_3days'] = (ontime_pgd['ontime']+ontime_pgd['paid_after_3_day'])/ontime_pgd['All']
ontime_pgd = ontime_pgd.sort_values(by=['ontime_rate'])
ontime_pgd_last_month['All']=ontime_pgd_last_month['ontime']+ontime_pgd_last_month['paid_after_3_day']+ontime_pgd_last_month['still_over_due']
ontime_pgd_last_month['ontime_rate'] = ontime_pgd_last_month['ontime']/ontime_pgd_last_month['All']
ontime_pgd_last_month['pmt_after_3days'] = (ontime_pgd_last_month['ontime']+ontime_pgd_last_month['paid_after_3_day'])/ontime_pgd_last_month['All']
ontime_pgd_last_month = ontime_pgd_last_month.sort_values(by=['ontime_rate'])

In [23]:
ontime_pgd_motor_title = ontime_pgd[ontime_pgd['categoryname'] == 'Đăng ký xe máy'] 
ontime_pgd_motor_title_last_month = ontime_pgd_last_month[ontime_pgd_last_month['categoryname'] == 'Đăng ký xe máy'] 
ontime_pgd_car_title = ontime_pgd[ontime_pgd['categoryname'] == 'Đăng ký Ô tô'] 
ontime_pgd_car_title_last_month = ontime_pgd_last_month[ontime_pgd_last_month['categoryname'] == 'Đăng ký Ô tô'] 

In [24]:
ontime_pgd_motor_title

stt_after_3_day,categoryname,khu_vuc,shopname,ontime,out_of_range,paid_after_3_day,still_over_due,All,ontime_rate,pmt_after_3days
160,Đăng ký xe máy,Tỉnh MB,QNH0206.81 Vĩnh Thông,57.0,3.0,26.0,17.0,100.0,0.570000,0.830000
159,Đăng ký xe máy,Tỉnh MB,QNH0192.670 Hạ Long,51.0,0.0,19.0,14.0,84.0,0.607143,0.833333
136,Đăng ký xe máy,Tỉnh MB,F88 BN - 216 Ngô Gia Tự,168.0,5.0,52.0,54.0,274.0,0.613139,0.802920
168,Đăng ký xe máy,Tỉnh MB,THA0187.342 Nguyễn Trãi,87.0,1.0,29.0,21.0,137.0,0.635036,0.846715
45,Đăng ký xe máy,Hà Nội,HNI0134.112 Phố Ga,68.0,4.0,27.0,12.0,107.0,0.635514,0.887850
134,Đăng ký xe máy,Tỉnh MB,BNH0235.Ngã Tư Đông Côi,51.0,0.0,20.0,9.0,80.0,0.637500,0.887500
175,Đăng ký xe máy,Tỉnh MB,TNN0200.100 Trường Chinh,116.0,7.0,44.0,21.0,181.0,0.640884,0.883978
139,Đăng ký xe máy,Tỉnh MB,F88 TH - 01 Ngô Quyền,185.0,9.0,61.0,40.0,286.0,0.646853,0.860140
140,Đăng ký xe máy,Tỉnh MB,F88 TH - 349 Trần Phú,179.0,9.0,56.0,41.0,276.0,0.648551,0.851449
132,Đăng ký xe máy,Tỉnh MB,"BNH0182.01 Khu 1, Thị trấn Phố Mới",132.0,6.0,37.0,34.0,203.0,0.650246,0.832512


In [25]:
# ASM, category
ontime_qlkv = ontime_spdk.pivot_table(index = ['categoryname', 'QLKV'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_qlkv_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname', 'QLKV'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_qlkv = ontime_qlkv.fillna(0)
ontime_qlkv_last_month = ontime_qlkv_last_month.fillna(0)

In [26]:
# ontime_qlkv['all'] = ontime_qlkv['ontime']+ontime_qlkv['paid_after_3_day']
# ontime_qlkv['ontime_rate'] = ontime_qlkv['ontime']/ontime_qlkv['all']
# ontime_qlkv['paid_within_3_days_rate'] = (ontime_qlkv['ontime']+ontime_qlkv['paid_after_3_day'])/ontime_qlkv['all']
# ontime_qlkv_last_month['all'] = ontime_qlkv_last_month['ontime']+ontime_qlkv_last_month['paid_after_3_day']+ontime_qlkv_last_month['still_over_due']
# ontime_qlkv_last_month['ontime_rate'] = ontime_qlkv_last_month['ontime']/ontime_qlkv_last_month['all']
# ontime_qlkv_last_month['paid_within_3_days_rate'] = (ontime_qlkv_last_month['ontime']+ontime_qlkv_last_month['paid_after_3_day'])/ontime_qlkv_last_month['all']

In [27]:
ontime_qlkv['all'] = ontime_qlkv['ontime']+ontime_qlkv['paid_after_3_day']+ontime_qlkv['still_over_due']
ontime_qlkv['ontime_rate'] = ontime_qlkv['ontime']/ontime_qlkv['all']
ontime_qlkv['paid_within_3_days_rate'] = (ontime_qlkv['ontime']+ontime_qlkv['paid_after_3_day'])/ontime_qlkv['all']
ontime_qlkv_last_month['all'] = ontime_qlkv_last_month['ontime']+ontime_qlkv_last_month['paid_after_3_day']+ontime_qlkv_last_month['still_over_due']
ontime_qlkv_last_month['ontime_rate'] = ontime_qlkv_last_month['ontime']/ontime_qlkv_last_month['all']
ontime_qlkv_last_month['paid_within_3_days_rate'] = (ontime_qlkv_last_month['ontime']+ontime_qlkv_last_month['paid_after_3_day'])/ontime_qlkv_last_month['all']

In [28]:
ontime_qlkv_motor_title = ontime_qlkv[ontime_qlkv['categoryname'] == 'Đăng ký xe máy']
ontime_qlkv_car_title = ontime_qlkv[ontime_qlkv['categoryname'] == 'Đăng ký Ô tô']
ontime_qlkv_motor_title_last_month = ontime_qlkv_last_month[ontime_qlkv_last_month['categoryname'] == 'Đăng ký xe máy']
ontime_qlkv_car_title_last_month = ontime_qlkv_last_month[ontime_qlkv_last_month['categoryname'] == 'Đăng ký Ô tô']

In [29]:
#  category, province
ontime_province_motor_title = ontime_dkxm_this_month.pivot_table(index = 'Tỉnh/ Thành phố', columns= 'stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_province_car_title = ontime_dkot_this_month.pivot_table(index = 'Tỉnh/ Thành phố', columns= 'stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_province_motor_title_last_month = ontime_dkxm_last_month.pivot_table(index = 'Tỉnh/ Thành phố', columns= 'stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_province_car_title_last_month = ontime_dkot_last_month.pivot_table(index = 'Tỉnh/ Thành phố', columns= 'stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()

In [30]:
ontime_province_motor_title = ontime_province_motor_title.fillna(0)
ontime_province_car_title = ontime_province_car_title.fillna(0)
ontime_province_motor_title_last_month = ontime_province_motor_title_last_month.fillna(0)
ontime_province_car_title_last_month = ontime_province_car_title_last_month.fillna(0)

In [31]:
ontime_province_motor_title['total'] = ontime_province_motor_title['ontime']+ontime_province_motor_title['paid_after_3_day']
# ontime_province_motor_title['total'] = ontime_province_motor_title['ontime']+ontime_province_motor_title['paid_after_3_day']+ontime_province_motor_title['still_over_due']

In [32]:
# ontime_province_car_title['total'] = ontime_province_car_title['ontime']+ontime_province_car_title['paid_after_3_day']
# ontime_province_motor_title_last_month['total'] = ontime_province_motor_title_last_month['ontime']+ontime_province_motor_title_last_month['paid_after_3_day']
# ontime_province_car_title_last_month['total'] = ontime_province_car_title_last_month['ontime']+ontime_province_car_title_last_month['paid_after_3_day']

In [33]:
ontime_province_car_title['total'] = ontime_province_car_title['ontime']+ontime_province_car_title['paid_after_3_day']+ontime_province_car_title['still_over_due']
ontime_province_motor_title_last_month['total'] = ontime_province_motor_title_last_month['ontime']+ontime_province_motor_title_last_month['paid_after_3_day']+ontime_province_motor_title_last_month['still_over_due']
ontime_province_car_title_last_month['total'] = ontime_province_car_title_last_month['ontime']+ontime_province_car_title_last_month['paid_after_3_day']+ontime_province_car_title_last_month['still_over_due']

In [34]:
ontime_province_motor_title['ontime_rate'] = ontime_province_motor_title['ontime']/ontime_province_motor_title['total']
ontime_province_motor_title_last_month['ontime_rate'] = ontime_province_motor_title_last_month['ontime']/ontime_province_motor_title_last_month['total']
ontime_province_car_title['ontime_rate'] = ontime_province_car_title['ontime']/ontime_province_car_title['total']
ontime_province_car_title_last_month['ontime_rate'] = ontime_province_car_title_last_month['ontime']/ontime_province_car_title_last_month['total']
ontime_province_motor_title['pmt_within_3days_rate'] = (ontime_province_motor_title['ontime']+ontime_province_motor_title['paid_after_3_day'])/ontime_province_motor_title['total']
ontime_province_motor_title_last_month['pmt_within_3days_rate'] = (ontime_province_motor_title_last_month['ontime']+ontime_province_motor_title_last_month['paid_after_3_day'])/ontime_province_motor_title_last_month['total']
ontime_province_car_title['pmt_within_3days_rate'] =(ontime_province_car_title['ontime']+ontime_province_car_title['paid_after_3_day'])/ontime_province_car_title['total']
ontime_province_car_title_last_month['pmt_within_3days_rate'] = (ontime_province_car_title_last_month['ontime']+ontime_province_car_title_last_month['paid_after_3_day'])/ontime_province_car_title_last_month['total']

In [35]:
# scheduletodate, category
ontime_schedule = ontime_spdk.pivot_table(index = 'scheduletodate', columns= ['categoryname', 'pmt_stt'], values='pawnid', aggfunc='nunique').reset_index()
ontime_schedule_last_month = ontime_spdk_last_month.pivot_table(index = 'scheduletodate', columns= ['categoryname', 'pmt_stt'], values='pawnid', aggfunc='nunique').reset_index()
ontime_schedule = ontime_schedule.fillna(0)
ontime_schedule_last_month = ontime_schedule_last_month.fillna(0)

In [36]:
ontime_spdk.head(5)

,GĐM,Kinh độ,Mã PGD,Phường/Xã,QLKV,Quận/Huyện,Shop ID,TPK,Thời gian KT,Tên PGD,...,row_num,scheduletodate,shopcode,shopname,status,stt_after_3_day,t_3_day_after,title_type,todate,write_off_period
0,Lê Ngọ,106.759579,SG2022,P. Bình Trưng Tây,Võ Minh Triều,Q. 2,205,Lý Quốc Huy,2019 - 5,F88 HCM - 143 Nguyễn Duy Trinh,...,0.0,2020-12-27,SG2022,F88 HCM - 143 Nguyễn Duy Trinh,14,out_of_range,2020-12-30,monthly_org_paid,2021-07-25,0.0
1,Lê Ngọ,106.610791,SG2042,X. Trung Chánh,Nguyễn Đình Nhật,H. Hóc Môn,227,Trần Thanh Phú,2019 - 7,F88 HCM - 128/1 Nguyễn Ảnh Thủ,...,0.0,2020-12-04,SG2042,F88 HCM - 128/1 Nguyễn Ảnh Thủ,14,ontime,2020-12-07,monthly_org_paid,2021-08-01,0.0
2,Phan Trung Kiên,105.778950,THH,P. Điện Biên,Trần Đức Tiến,TP. Thanh Hóa,34,Nguyễn Mạnh Tùng,2016 - 10,F88 TH - 01 Ngô Quyền,...,0.0,2020-12-23,THH,F88 TH - 01 Ngô Quyền,14,still_over_due,2020-12-26,monthly_org_paid,2021-09-19,0.0
3,Lê Ngọ,106.648587,SG2003,P. 10,Lý Văn Đạt,Q. Tân Bình,183,Trần Thanh Phú,2018 - 11,F88 HCM - 656 Lạc Long Quân,...,0.0,2020-12-19,SG2003,F88 HCM - 656 Lạc Long Quân,14,ontime,2020-12-22,monthly_org_paid,2021-10-15,0.0
4,Lê Ngọ,106.635845,SG2051,P. Phú Thạnh,Phạm Hữu Thành,Q. Tân Phú,238,Trần Thanh Phú,2019 - 9,F88 HCM - 683 Lũy Bán Bích,...,0.0,2020-12-20,SG2051,F88 HCM - 683 Lũy Bán Bích,14,paid_after_3_day,2020-12-23,monthly_org_paid,2021-11-15,0.0


In [37]:
# scheduletodate, category
ontime_schedule_3 = ontime_spdk.pivot_table(index = 'scheduletodate', columns= ['categoryname', 'stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()
ontime_schedule_3_last_month = ontime_spdk_last_month.pivot_table(index = 'scheduletodate', columns= ['categoryname', 'stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()
ontime_schedule_3 = ontime_schedule_3.fillna(0)
ontime_schedule_3_last_month = ontime_schedule_3_last_month.fillna(0)

In [38]:
ontime_schedule_3

categoryname    scheduletodate Đăng ký xe máy                                \
stt_after_3_day                        ontime out_of_range paid_after_3_day   
0                   2020-12-01         1146.0          0.0            207.0   
1                   2020-12-02          833.0          0.0            153.0   
2                   2020-12-03          848.0          0.0            155.0   
3                   2020-12-04          853.0          0.0            124.0   
4                   2020-12-05         1506.0          0.0            279.0   
5                   2020-12-06         1028.0          0.0            220.0   
6                   2020-12-07         1097.0          0.0            189.0   
7                   2020-12-08          955.0          0.0            171.0   
8                   2020-12-09          954.0          0.0            173.0   
9                   2020-12-10         2384.0          0.0            421.0   
10                  2020-12-11         1294.0          0.0            209.0   
11                  2020-12-12         1658.0          0.0            329.0   
12                  2020-12-13         1068.0          0.0            204.0   
13                  2020-12-14         1083.0          0.0            218.0   
14                  2020-12-15         3072.0          0.0            613.0   
15                  2020-12-16         1533.0          0.0            293.0   
16                  2020-12-17         1422.0          0.0            216.0   
17                  2020-12-18         1270.0          0.0            240.0   
18                  2020-12-19         1083.0          0.0            219.0   
19                  2020-12-20         2123.0          0.0            509.0   
20                  2020-12-21         1128.0          0.0            238.0   
21                  2020-12-22         1126.0          0.0            218.0   
22                  2020-12-23         1056.0          0.0            184.0   
23                  2020-12-24          977.0          0.0            196.0   
24                  2020-12-25         1718.0          0.0            303.0   
25                  2020-12-26         1137.0          0.0            345.0   
26                  2020-12-27         1008.0        375.0              0.0   
27                  2020-12-28         1572.0        605.0              0.0   

categoryname                   Đăng ký Ô tô                                \
stt_after_3_day still_over_due       ontime out_of_range paid_after_3_day   
0                        151.0         19.0          0.0              9.0   
1                        139.0         28.0          0.0             10.0   
2                        148.0         23.0          0.0              8.0   
3                        136.0         24.0          0.0             12.0   
4                        233.0         35.0          0.0             10.0   
5                        154.0         29.0          0.0             15.0   
6                        131.0         29.0          0.0             12.0   
7                        123.0         26.0          0.0             15.0   
8                        127.0         23.0          0.0             12.0   
9                        337.0         20.0          0.0              8.0   
10                       177.0         27.0          0.0              8.0   
11                       228.0         30.0          0.0             18.0   
12                       135.0         33.0          0.0             11.0   
13                       150.0         24.0          0.0              9.0   
14                       471.0         35.0          0.0             10.0   
15                       213.0         21.0          0.0              9.0   
16                       219.0         27.0          0.0             16.0   
17                       192.0         18.0          0.0              4.0   
18                       161.0         36.0          0.0             11.0   
19          

In [39]:
# scheduletodate, package_type, cat
# scheduletodate, category
ontime_schedule_4 = ontime_spdk.pivot_table(index = ['package_type','scheduletodate'], columns= ['categoryname', 'stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()
ontime_schedule_4_last_month = ontime_spdk_last_month.pivot_table(index = ['package_type','scheduletodate'], columns= ['categoryname', 'stt_after_3_day'], values='pawnid', aggfunc='nunique').reset_index()
ontime_schedule_4 = ontime_schedule_4.fillna(0)
ontime_schedule_4_last_month = ontime_schedule_4_last_month.fillna(0)

In [40]:
ontime_schedule_4_vaynhanh = ontime_schedule_4[ontime_schedule_4['package_type'] == 'Vay nhanh']

In [41]:
ontime_schedule_4_vaythuong = ontime_schedule_4[ontime_schedule_4['package_type'] != 'Vay nhanh']

In [42]:
ontime_schedule_4_vaynhanh_last_month = ontime_schedule_4_last_month[ontime_schedule_4_last_month['package_type'] == 'Vay nhanh']

In [43]:
ontime_schedule_4_vaythuong_last_month = ontime_schedule_4_last_month[ontime_schedule_4_last_month['package_type'] != 'Vay nhanh']

In [44]:
# categoryname, scheduletodate, day in month
ontime_spdk['day_in_month_sheduletodate'] = ontime_spdk['scheduletodate'].astype('datetime64[ns]').dt.day
ontime_spdk_last_month['day_in_month_sheduletodate'] = ontime_spdk_last_month['scheduletodate'].astype('datetime64[ns]').dt.day
ontime_by_day_category = ontime_spdk.pivot_table(index = 'day_in_month_sheduletodate', columns=['categoryname','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_by_day_category = ontime_by_day_category.fillna(0)
ontime_by_day_category_last_month = ontime_spdk_last_month.pivot_table(index = 'day_in_month_sheduletodate', columns=['categoryname','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_by_day_category_last_month = ontime_by_day_category_last_month.fillna(0)

In [45]:
pmt_3days_by_day_category = ontime_spdk.pivot_table(index = 'day_in_month_sheduletodate', columns=['categoryname','stt_after_3_day'], values='pawnid', aggfunc='count').reset_index()
pmt_3days_by_day_category = ontime_by_day_category.fillna(0)
pmt_3days_by_day_category_last_month = ontime_spdk_last_month.pivot_table(index = 'day_in_month_sheduletodate', columns=['categoryname','stt_after_3_day'], values='pawnid', aggfunc='count').reset_index()
pmt_3days_by_day_category_last_month = ontime_by_day_category_last_month.fillna(0)

In [46]:
pmt_3days_by_day_cate = ontime_spdk.pivot_table(index = 'day_in_month_sheduletodate', columns=['categoryname','stt_after_3_day'], values='pawnid', aggfunc='count').reset_index()

In [47]:
pmt_3days_by_day_cate = pmt_3days_by_day_cate.fillna(0)
pmt_3days_by_day_cate_last_month = ontime_spdk_last_month.pivot_table(index = 'day_in_month_sheduletodate', columns=['categoryname','stt_after_3_day'], values='pawnid', aggfunc='count').reset_index()
pmt_3days_by_day_cate_last_month = pmt_3days_by_day_cate_last_month.fillna(0)

In [48]:
# category, scheduletodate, area
ontime_schedule_area_motortitle = ontime_dkxm_this_month.pivot_table(index = 'scheduletodate', columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_motortitle_last_month = ontime_dkxm_last_month.pivot_table(index = 'scheduletodate', columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_cartitle = ontime_dkot_this_month.pivot_table(index = 'scheduletodate', columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_cartitle_last_month = ontime_dkot_last_month.pivot_table(index = 'scheduletodate', columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()

In [49]:
ontime_schedule_area_motortitle = ontime_schedule_area_motortitle.fillna(0)
ontime_schedule_area_motortitle_last_month = ontime_schedule_area_motortitle_last_month.fillna(0)
ontime_schedule_area_cartitle = ontime_schedule_area_cartitle.fillna(0)
ontime_schedule_area_cartitle_last_month = ontime_schedule_area_cartitle_last_month.fillna(0)

In [50]:
# category, scheduletodate, area, package_type
ontime_schedule_area_motortitle_package = ontime_dkxm_this_month.pivot_table(index = ['package_type','scheduletodate'], columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_motortitle_package_last_month = ontime_dkxm_last_month.pivot_table(index =  ['package_type','scheduletodate'], columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_cartitle_package = ontime_dkot_this_month.pivot_table(index =  ['package_type','scheduletodate'], columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_cartitle_package_last_month = ontime_dkot_last_month.pivot_table(index =  ['package_type','scheduletodate'], columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_schedule_area_motortitle_package = ontime_schedule_area_motortitle_package.fillna(0)
ontime_schedule_area_motortitle_package_last_month = ontime_schedule_area_motortitle_package_last_month.fillna(0)
ontime_schedule_area_cartitle_package = ontime_schedule_area_cartitle_package.fillna(0)
ontime_schedule_area_cartitle_package_last_month = ontime_schedule_area_cartitle_package_last_month.fillna(0)

In [51]:
vn_ontime_schedule_area_motortitle_package = ontime_schedule_area_motortitle_package[ontime_schedule_area_motortitle_package['package_type']=='Vay nhanh']
vt_ontime_schedule_area_motortitle_package = ontime_schedule_area_motortitle_package[ontime_schedule_area_motortitle_package['package_type']=='Vay thường']

vn_ontime_schedule_area_motortitle_package_last_month = ontime_schedule_area_motortitle_package_last_month[ontime_schedule_area_motortitle_package_last_month['package_type']=='Vay nhanh']
vt_ontime_schedule_area_motortitle_package_last_month = ontime_schedule_area_motortitle_package_last_month[ontime_schedule_area_motortitle_package_last_month['package_type']=='Vay thường']

vn_ontime_schedule_area_cartitle_package = ontime_schedule_area_cartitle_package[ontime_schedule_area_cartitle_package['package_type']=='Vay nhanh']
vt_ontime_schedule_area_cartitle_package = ontime_schedule_area_cartitle_package[ontime_schedule_area_cartitle_package['package_type']=='Vay thường']

vn_ontime_schedule_area_cartitle_package_last_month = ontime_schedule_area_cartitle_package_last_month[ontime_schedule_area_cartitle_package_last_month['package_type']=='Vay nhanh']
vt_ontime_schedule_area_cartitle_package_last_month = ontime_schedule_area_cartitle_package_last_month[ontime_schedule_area_cartitle_package_last_month['package_type']=='Vay thường']

In [52]:
# category, scheduletodate, area, package

In [53]:
# ontime_brr_ym_overview_motor_title = ontime_dkxm_this_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
# ontime_brr_ym_overview_motor_title = ontime_brr_ym_overview_motor_title.fillna(0)
# ontime_brr_ym_overview_motor_title['all'] = ontime_brr_ym_overview_motor_title['ontime']+ontime_brr_ym_overview_motor_title['paid_after_3_day']
# ontime_brr_ym_overview_motor_title['ontime_rate'] = ontime_brr_ym_overview_motor_title['ontime']/ontime_brr_ym_overview_motor_title['all']
# ontime_brr_ym_overview_motor_title['paid_within_3_days_rate'] = (ontime_brr_ym_overview_motor_title['ontime']+ontime_brr_ym_overview_motor_title['paid_after_3_day'])/ontime_brr_ym_overview_motor_title['all']

In [54]:
# brr year month, category - motor title
ontime_brr_ym_overview_motor_title = ontime_dkxm_this_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_brr_ym_overview_motor_title = ontime_brr_ym_overview_motor_title.fillna(0)
ontime_brr_ym_overview_motor_title['all'] = ontime_brr_ym_overview_motor_title['ontime']+ontime_brr_ym_overview_motor_title['paid_after_3_day']+ontime_brr_ym_overview_motor_title['still_over_due']
ontime_brr_ym_overview_motor_title['ontime_rate'] = ontime_brr_ym_overview_motor_title['ontime']/ontime_brr_ym_overview_motor_title['all']
ontime_brr_ym_overview_motor_title['paid_within_3_days_rate'] = (ontime_brr_ym_overview_motor_title['ontime']+ontime_brr_ym_overview_motor_title['paid_after_3_day'])/ontime_brr_ym_overview_motor_title['all']

In [55]:
# # brr year month, category - car title
# ontime_brr_ym_overview_car_title = ontime_dkot_this_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
# ontime_brr_ym_overview_car_title = ontime_brr_ym_overview_car_title.fillna(0)
# ontime_brr_ym_overview_car_title['all'] = ontime_brr_ym_overview_car_title['ontime']+ontime_brr_ym_overview_car_title['paid_after_3_day']
# ontime_brr_ym_overview_car_title['ontime_rate'] = ontime_brr_ym_overview_car_title['ontime']/ontime_brr_ym_overview_car_title['all']
# ontime_brr_ym_overview_car_title['paid_within_3_days_rate'] = (ontime_brr_ym_overview_car_title['ontime']+ontime_brr_ym_overview_car_title['paid_after_3_day'])/ontime_brr_ym_overview_car_title['all']

In [56]:
# brr year month, category - car title
ontime_brr_ym_overview_car_title = ontime_dkot_this_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_brr_ym_overview_car_title = ontime_brr_ym_overview_car_title.fillna(0)
ontime_brr_ym_overview_car_title['all'] = ontime_brr_ym_overview_car_title['ontime']+ontime_brr_ym_overview_car_title['paid_after_3_day']+ontime_brr_ym_overview_car_title['still_over_due']
ontime_brr_ym_overview_car_title['ontime_rate'] = ontime_brr_ym_overview_car_title['ontime']/ontime_brr_ym_overview_car_title['all']
ontime_brr_ym_overview_car_title['paid_within_3_days_rate'] = (ontime_brr_ym_overview_car_title['ontime']+ontime_brr_ym_overview_car_title['paid_after_3_day'])/ontime_brr_ym_overview_car_title['all']

In [57]:
# # brr year month, category - motor title last month
# ontime_brr_ym_overview_motor_title_last_month = ontime_dkxm_last_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
# ontime_brr_ym_overview_motor_title_last_month = ontime_brr_ym_overview_motor_title_last_month.fillna(0)
# ontime_brr_ym_overview_motor_title_last_month['all'] = ontime_brr_ym_overview_motor_title_last_month['ontime']+ontime_brr_ym_overview_motor_title_last_month['paid_after_3_day']
# ontime_brr_ym_overview_motor_title_last_month['ontime_rate'] = ontime_brr_ym_overview_motor_title_last_month['ontime']/ontime_brr_ym_overview_motor_title_last_month['all']
# ontime_brr_ym_overview_motor_title_last_month['paid_within_3_days_rate'] = (ontime_brr_ym_overview_motor_title_last_month['ontime']+ontime_brr_ym_overview_motor_title_last_month['paid_after_3_day'])/ontime_brr_ym_overview_motor_title_last_month['all']

In [58]:
# brr year month, category - motor title last month
ontime_brr_ym_overview_motor_title_last_month = ontime_dkxm_last_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_brr_ym_overview_motor_title_last_month = ontime_brr_ym_overview_motor_title_last_month.fillna(0)
ontime_brr_ym_overview_motor_title_last_month['all'] = ontime_brr_ym_overview_motor_title_last_month['ontime']+ontime_brr_ym_overview_motor_title_last_month['paid_after_3_day']+ontime_brr_ym_overview_motor_title_last_month['still_over_due']
ontime_brr_ym_overview_motor_title_last_month['ontime_rate'] = ontime_brr_ym_overview_motor_title_last_month['ontime']/ontime_brr_ym_overview_motor_title_last_month['all']
ontime_brr_ym_overview_motor_title_last_month['paid_within_3_days_rate'] = (ontime_brr_ym_overview_motor_title_last_month['ontime']+ontime_brr_ym_overview_motor_title_last_month['paid_after_3_day'])/ontime_brr_ym_overview_motor_title_last_month['all']

In [59]:
# # brr year month, category - car title last month
# ontime_brr_ym_overview_car_title_last_month = ontime_dkot_last_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
# ontime_brr_ym_overview_car_title_last_month = ontime_brr_ym_overview_car_title_last_month.fillna(0)
# ontime_brr_ym_overview_car_title_last_month['all'] = ontime_brr_ym_overview_car_title_last_month['ontime']+ontime_brr_ym_overview_car_title_last_month['paid_after_3_day']
# ontime_brr_ym_overview_car_title_last_month['ontime_rate'] = ontime_brr_ym_overview_car_title_last_month['ontime']/ontime_brr_ym_overview_car_title_last_month['all']
# ontime_brr_ym_overview_car_title_last_month['paid_within_3_days_rate'] = (ontime_brr_ym_overview_car_title_last_month['ontime']+ontime_brr_ym_overview_car_title_last_month['paid_after_3_day'])/ontime_brr_ym_overview_car_title_last_month['all']

In [60]:
# brr year month, category - car title last month
ontime_brr_ym_overview_car_title_last_month = ontime_dkot_last_month.pivot_table(index = 'brr_ym', columns='stt_after_3_day', values='pawnid', aggfunc='nunique').reset_index()
ontime_brr_ym_overview_car_title_last_month = ontime_brr_ym_overview_car_title_last_month.fillna(0)
ontime_brr_ym_overview_car_title_last_month['all'] = ontime_brr_ym_overview_car_title_last_month['ontime']+ontime_brr_ym_overview_car_title_last_month['paid_after_3_day']+ontime_brr_ym_overview_car_title_last_month['still_over_due']
ontime_brr_ym_overview_car_title_last_month['ontime_rate'] = ontime_brr_ym_overview_car_title_last_month['ontime']/ontime_brr_ym_overview_car_title_last_month['all']
ontime_brr_ym_overview_car_title_last_month['paid_within_3_days_rate'] = (ontime_brr_ym_overview_car_title_last_month['ontime']+ontime_brr_ym_overview_car_title_last_month['paid_after_3_day'])/ontime_brr_ym_overview_car_title_last_month['all']

In [61]:
#  brr_ym, category, package
ontime_category_brr_ym_package_motor_title = ontime_dkxm_this_month.pivot_table(index='brr_ym', columns=['package_type', 'pmt_stt'], values ='pawnid', aggfunc ='nunique').reset_index()
ontime_category_brr_ym_package_motor_title = ontime_category_brr_ym_package_motor_title.fillna(0)
ontime_category_brr_ym_package_car_title = ontime_dkot_this_month.pivot_table(index='brr_ym', columns=['package_type', 'pmt_stt'], values ='pawnid', aggfunc ='nunique').reset_index()
ontime_category_brr_ym_package_car_title = ontime_category_brr_ym_package_car_title.fillna(0)

In [62]:
ontime_category_brr_ym_package_motor_title_last_month = ontime_dkxm_last_month.pivot_table(index='brr_ym', columns=['package_type', 'pmt_stt'], values ='pawnid', aggfunc ='nunique').reset_index()
ontime_category_brr_ym_package_motor_title_last_month = ontime_category_brr_ym_package_motor_title_last_month.fillna(0)
ontime_category_brr_ym_package_car_title_last_month = ontime_dkot_last_month.pivot_table(index='brr_ym', columns=['package_type', 'pmt_stt'], values ='pawnid', aggfunc ='nunique').reset_index()
ontime_category_brr_ym_package_car_title_last_month = ontime_category_brr_ym_package_car_title_last_month.fillna(0)

In [63]:
# # category, package, money range
# ontime_money_range_package = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
# ontime_money_range_package = ontime_money_range_package.fillna(0)
# ontime_money_range_package['all'] = ontime_money_range_package['ontime']+ontime_money_range_package['paid_after_3_day']
# ontime_money_range_package['ontime_rate'] = ontime_money_range_package['ontime']/ontime_money_range_package['all']
# ontime_money_range_package['paid_within_3_days_rate'] = (ontime_money_range_package['ontime']+ontime_money_range_package['paid_after_3_day'])/ontime_money_range_package['all']

In [64]:
# category, package, money range
ontime_money_range_package = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
ontime_money_range_package = ontime_money_range_package.fillna(0)
ontime_money_range_package['all'] = ontime_money_range_package['ontime']+ontime_money_range_package['paid_after_3_day']+ontime_money_range_package['still_over_due']
ontime_money_range_package['ontime_rate'] = ontime_money_range_package['ontime']/ontime_money_range_package['all']
ontime_money_range_package['paid_within_3_days_rate'] = (ontime_money_range_package['ontime']+ontime_money_range_package['paid_after_3_day'])/ontime_money_range_package['all']

In [65]:
# ontime_money_range_package_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
# ontime_money_range_package_last_month = ontime_money_range_package_last_month.fillna(0)
# ontime_money_range_package_last_month['all'] = ontime_money_range_package_last_month['ontime']+ontime_money_range_package_last_month['paid_after_3_day']
# ontime_money_range_package_last_month['ontime_rate'] = ontime_money_range_package_last_month['ontime']/ontime_money_range_package_last_month['all']
# ontime_money_range_package_last_month['paid_within_3_days_rate'] = (ontime_money_range_package_last_month['ontime']+ontime_money_range_package_last_month['paid_after_3_day'])/ontime_money_range_package_last_month['all']

In [66]:
ontime_money_range_package_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
ontime_money_range_package_last_month = ontime_money_range_package_last_month.fillna(0)
ontime_money_range_package_last_month['all'] = ontime_money_range_package_last_month['ontime']+ontime_money_range_package_last_month['paid_after_3_day']+ontime_money_range_package_last_month['still_over_due']
ontime_money_range_package_last_month['ontime_rate'] = ontime_money_range_package_last_month['ontime']/ontime_money_range_package_last_month['all']
ontime_money_range_package_last_month['paid_within_3_days_rate'] = (ontime_money_range_package_last_month['ontime']+ontime_money_range_package_last_month['paid_after_3_day'])/ontime_money_range_package_last_month['all']

In [67]:
# # category, package, time range
# ontime_time_range_package = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_time_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
# ontime_time_range_package = ontime_time_range_package.fillna(0)
# ontime_time_range_package['all'] = ontime_time_range_package['ontime']+ontime_time_range_package['paid_after_3_day']
# ontime_time_range_package['ontime_rate'] = ontime_time_range_package['ontime']/ontime_time_range_package['all']
# ontime_time_range_package['paid_within_3_days_rate'] = (ontime_time_range_package['ontime']+ontime_time_range_package['paid_after_3_day'])/ontime_time_range_package['all']

In [68]:
# category, package, time range
ontime_time_range_package = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_time_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
ontime_time_range_package = ontime_time_range_package.fillna(0)
ontime_time_range_package['all'] = ontime_time_range_package['ontime']+ontime_time_range_package['paid_after_3_day']+ontime_time_range_package['still_over_due']
ontime_time_range_package['ontime_rate'] = ontime_time_range_package['ontime']/ontime_time_range_package['all']
ontime_time_range_package['paid_within_3_days_rate'] = (ontime_time_range_package['ontime']+ontime_time_range_package['paid_after_3_day'])/ontime_time_range_package['all']

In [69]:
# ontime_time_range_package_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_time_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
# ontime_time_range_package_last_month = ontime_time_range_package_last_month.fillna(0)
# ontime_time_range_package_last_month['all'] = ontime_time_range_package_last_month['ontime']+ontime_time_range_package_last_month['paid_after_3_day']
# ontime_time_range_package_last_month['ontime_rate'] = ontime_time_range_package_last_month['ontime']/ontime_time_range_package_last_month['all']
# ontime_time_range_package_last_month['paid_within_3_days_rate'] = (ontime_time_range_package_last_month['ontime']+ontime_time_range_package_last_month['paid_after_3_day'])/ontime_time_range_package_last_month['all']

In [70]:
ontime_time_range_package_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_time_range'], columns= 'stt_after_3_day', values='pawnid',aggfunc ='nunique').reset_index()
ontime_time_range_package_last_month = ontime_time_range_package_last_month.fillna(0)
ontime_time_range_package_last_month['all'] = ontime_time_range_package_last_month['ontime']+ontime_time_range_package_last_month['paid_after_3_day']+ontime_time_range_package_last_month['still_over_due']
ontime_time_range_package_last_month['ontime_rate'] = ontime_time_range_package_last_month['ontime']/ontime_time_range_package_last_month['all']
ontime_time_range_package_last_month['paid_within_3_days_rate'] = (ontime_time_range_package_last_month['ontime']+ontime_time_range_package_last_month['paid_after_3_day'])/ontime_time_range_package_last_month['all']

In [71]:
# category, package, money range, area
ontime_money_range_package_area = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= ['khu_vuc','pmt_stt'], values='pawnid',aggfunc ='nunique').reset_index()
ontime_money_range_package_area = ontime_money_range_package_area.fillna(0)

In [72]:
ontime_money_range_package_area_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= ['khu_vuc','pmt_stt'], values='pawnid',aggfunc ='nunique').reset_index()
ontime_money_range_package_area_last_month = ontime_money_range_package_area_last_month.fillna(0)

In [73]:
# category, package, time range, area
ontime_time_range_package_area = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_time_range'], columns= ['khu_vuc','pmt_stt'], values='pawnid',aggfunc ='nunique').reset_index()
ontime_time_range_package_area = ontime_time_range_package_area.fillna(0)
ontime_time_range_package_area_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_money_range'], columns= ['khu_vuc','pmt_stt'], values='pawnid',aggfunc ='nunique').reset_index()
ontime_time_range_package_area_last_month = ontime_time_range_package_area_last_month.fillna(0)

In [74]:
#  category, money range, time range
ontime_money_time_range = ontime_spdk.pivot_table(index = ['categoryname','package_type','package_money_range'], columns=['package_time_range','pmt_stt'], values='pawnid',aggfunc ='nunique').reset_index()
ontime_money_time_range = ontime_money_time_range.fillna(0)

ontime_money_time_range_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type','package_money_range'], columns=['package_time_range','pmt_stt'], values='pawnid',aggfunc ='nunique').reset_index()
ontime_money_time_range_last_month = ontime_money_time_range_last_month.fillna(0)

In [75]:
# # custype
# ontime_cus_type_overview = ontime_spdk.pivot_table(index =  'cus_type', columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
# ontime_cus_type_overview_last_month = ontime_spdk_last_month.pivot_table(index =  'cus_type', columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
# ontime_cus_type_overview = ontime_cus_type_overview.fillna(0)
# ontime_cus_type_overview_last_month = ontime_cus_type_overview_last_month.fillna(0)
# ontime_cus_type_overview['all'] = ontime_cus_type_overview['ontime']+ontime_cus_type_overview['paid_after_3_day']
# ontime_cus_type_overview['ontime_rate'] = ontime_cus_type_overview['ontime']/ontime_cus_type_overview['all']
# ontime_cus_type_overview['paid_within_3days_rate'] = (ontime_cus_type_overview['ontime']+ontime_cus_type_overview['paid_after_3_day'])/ontime_cus_type_overview['all']
# ontime_cus_type_overview_last_month['all'] = ontime_cus_type_overview_last_month['ontime']+ontime_cus_type_overview_last_month['paid_after_3_day']
# ontime_cus_type_overview_last_month['ontime_rate'] = ontime_cus_type_overview_last_month['ontime']/ontime_cus_type_overview_last_month['all']
# ontime_cus_type_overview_last_month['paid_within_3days_rate'] = (ontime_cus_type_overview_last_month['ontime']+ontime_cus_type_overview_last_month['paid_after_3_day'])/ontime_cus_type_overview_last_month['all']

In [76]:
# custype
ontime_cus_type_overview = ontime_spdk.pivot_table(index =  'cus_type', columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_cus_type_overview_last_month = ontime_spdk_last_month.pivot_table(index =  'cus_type', columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_cus_type_overview = ontime_cus_type_overview.fillna(0)
ontime_cus_type_overview_last_month = ontime_cus_type_overview_last_month.fillna(0)
ontime_cus_type_overview['all'] = ontime_cus_type_overview['ontime']+ontime_cus_type_overview['paid_after_3_day']+ontime_cus_type_overview['still_over_due']
ontime_cus_type_overview['ontime_rate'] = ontime_cus_type_overview['ontime']/ontime_cus_type_overview['all']
ontime_cus_type_overview['paid_within_3days_rate'] = (ontime_cus_type_overview['ontime']+ontime_cus_type_overview['paid_after_3_day'])/ontime_cus_type_overview['all']
ontime_cus_type_overview_last_month['all'] = ontime_cus_type_overview_last_month['ontime']+ontime_cus_type_overview_last_month['paid_after_3_day']+ontime_cus_type_overview_last_month['still_over_due']
ontime_cus_type_overview_last_month['ontime_rate'] = ontime_cus_type_overview_last_month['ontime']/ontime_cus_type_overview_last_month['all']
ontime_cus_type_overview_last_month['paid_within_3days_rate'] = (ontime_cus_type_overview_last_month['ontime']+ontime_cus_type_overview_last_month['paid_after_3_day'])/ontime_cus_type_overview_last_month['all']

In [77]:
# # customer type, categoryname
# ontime_custype_category = ontime_spdk.pivot_table(index = ['categoryname', 'cus_type'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
# ontime_custype_category = ontime_custype_category.fillna(0)
# ontime_custype_category['all'] = ontime_custype_category['ontime']+ontime_custype_category['paid_after_3_day']
# ontime_custype_category['ontime_rate'] = ontime_custype_category['ontime']/ontime_custype_category['all']
# ontime_custype_category['paid_within_3days'] = (ontime_custype_category['ontime']+ontime_custype_category['paid_after_3_day'])/ontime_custype_category['all']

# ontime_custype_category_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname', 'cus_type'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
# ontime_custype_category_last_month = ontime_custype_category_last_month.fillna(0)
# ontime_custype_category_last_month['all'] = ontime_custype_category_last_month['ontime']+ontime_custype_category_last_month['paid_after_3_day']
# ontime_custype_category_last_month['ontime_rate'] = ontime_custype_category_last_month['ontime']/ontime_custype_category_last_month['all']
# ontime_custype_category_last_month['paid_within_3days'] = (ontime_custype_category_last_month['ontime']+ontime_custype_category_last_month['paid_after_3_day'])/ontime_custype_category_last_month['all']

In [78]:
# customer type, categoryname
ontime_custype_category = ontime_spdk.pivot_table(index = ['categoryname', 'cus_type'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_custype_category = ontime_custype_category.fillna(0)
ontime_custype_category['all'] = ontime_custype_category['ontime']+ontime_custype_category['paid_after_3_day']+ontime_custype_category['still_over_due']
ontime_custype_category['ontime_rate'] = ontime_custype_category['ontime']/ontime_custype_category['all']
ontime_custype_category['paid_within_3days'] = (ontime_custype_category['ontime']+ontime_custype_category['paid_after_3_day'])/ontime_custype_category['all']

ontime_custype_category_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname', 'cus_type'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_last_month = ontime_custype_category_last_month.fillna(0)
ontime_custype_category_last_month['all'] = ontime_custype_category_last_month['ontime']+ontime_custype_category_last_month['paid_after_3_day']+ontime_custype_category_last_month['still_over_due']
ontime_custype_category_last_month['ontime_rate'] = ontime_custype_category_last_month['ontime']/ontime_custype_category_last_month['all']
ontime_custype_category_last_month['paid_within_3days'] = (ontime_custype_category_last_month['ontime']+ontime_custype_category_last_month['paid_after_3_day'])/ontime_custype_category_last_month['all']

In [79]:
# customer type, categoryname, package
ontime_custype_category_package = ontime_spdk.pivot_table(index = ['categoryname','package_type', 'cus_type'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_package = ontime_custype_category_package.fillna(0)
ontime_custype_category_package['all'] = ontime_custype_category_package['ontime']+ontime_custype_category_package['paid_after_3_day']+ontime_custype_category_package['still_over_due']
ontime_custype_category_package['ontime_rate'] = ontime_custype_category_package['ontime']/ontime_custype_category_package['all']
ontime_custype_category_package['paid_within_3days'] = (ontime_custype_category_package['ontime']+ontime_custype_category_package['paid_after_3_day'])/ontime_custype_category_package['all']

ontime_custype_category_package_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','package_type', 'cus_type'], columns='stt_after_3_day', values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_package_last_month = ontime_custype_category_package_last_month.fillna(0)
ontime_custype_category_package_last_month['all'] = ontime_custype_category_package_last_month['ontime']+ontime_custype_category_package_last_month['paid_after_3_day']+ontime_custype_category_package_last_month['still_over_due']
ontime_custype_category_package_last_month['ontime_rate'] = ontime_custype_category_package_last_month['ontime']/ontime_custype_category_package_last_month['all']
ontime_custype_category_package_last_month['paid_within_3days'] = (ontime_custype_category_package_last_month['ontime']+ontime_custype_category_package_last_month['paid_after_3_day'])/ontime_custype_category_package_last_month['all']

In [80]:
# category, custype, area
ontime_custype_category_area = ontime_spdk.pivot_table(index = ['categoryname','cus_type'], columns=['khu_vuc','pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_area = ontime_custype_category_area.fillna(0)

ontime_custype_category_area_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','cus_type'], columns=['khu_vuc', 'pmt_stt'], values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_area_last_month = ontime_custype_category_area_last_month.fillna(0)

In [81]:
# category, custype, package
ontime_custype_category_area_package = ontime_spdk.pivot_table(index = ['categoryname','cus_type', 'package_type'], columns='pmt_stt', values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_area_package = ontime_custype_category_area_package.fillna(0)

ontime_custype_category_area_package_last_month = ontime_spdk_last_month.pivot_table(index = ['categoryname','cus_type', 'package_type'], columns= 'pmt_stt', values='pawnid', aggfunc='count').reset_index()
ontime_custype_category_area_package_last_month = ontime_custype_category_area_package_last_month.fillna(0)

In [82]:
ontime_custype_category_area_package['all'] =ontime_custype_category_area_package['ontime']+ontime_custype_category_area_package['overdue']
ontime_custype_category_area_package['ontime_rate'] =ontime_custype_category_area_package['ontime']/ontime_custype_category_area_package['all']

ontime_custype_category_area_package_last_month['all'] =ontime_custype_category_area_package_last_month['ontime']+ontime_custype_category_area_package_last_month['overdue']
ontime_custype_category_area_package_last_month['ontime_rate'] =ontime_custype_category_area_package_last_month['ontime']/ontime_custype_category_area_package_last_month['all']

In [83]:
# category, time measurement
ontime_spdk['final_payment_last_month'] = np.where(ontime_spdk['scheduletodate'] == ontime_spdk['todate'], 'y', 'n')
ontime_time_measurement = ontime_spdk.pivot_table(columns= ['final_payment_last_month','pmt_stt'], index = ['categoryname', 'khu_vuc'], values='pawnid', aggfunc='nunique')
ontime_time_measurement = ontime_time_measurement.reset_index()
ontime_time_measurement = ontime_time_measurement.fillna(0)

ontime_spdk_last_month['final_payment_last_month'] = np.where(ontime_spdk_last_month['scheduletodate'] == ontime_spdk_last_month['todate'], 'y', 'n')
ontime_time_measurement_last_month = ontime_spdk_last_month.pivot_table(columns= ['final_payment_last_month','pmt_stt'], index = ['categoryname', 'khu_vuc'], values='pawnid', aggfunc='nunique')
ontime_time_measurement_last_month = ontime_time_measurement_last_month.reset_index()
ontime_time_measurement_last_month = ontime_time_measurement_last_month.fillna(0)

REMIND

In [84]:
# motor title
def remind_rate(fromdate, todate):
    remind_str =  """Declare @schedule_from date ='"""+fromdate+"""'
                Declare @schedule_to date ='"""+todate+"""'
select tab.*,cmt.*,DATEDIFF(day,cmtdate,ScheduleToDate) day_cmt_before_schedule,
  case when cmtdate is null then N'1.no_cmt'
  when DATEDIFF(day,cmtdate,ScheduleToDate)=0 then N'3.on_due_date'
   when DATEDIFF(day,cmtdate,ScheduleToDate)<>0 then N'2.early_remind' end as cmt_type,
 case when PackageCode like '%NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
 from ( select  pd.pawnid,po.tuoino dpd_yesterday,
 pa.PackageCode,pa.shopname, pa.shopcode,
 areaid,pd.scheduletodate,pd.principalmoney,pd.acc_sum,pd.paydate, pd.acc_prin_payment,paidmoney,
 pa.status,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,   
  case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=acc_sum) or CloseDate<=ScheduleToDate 
   then N'ontime' else N'overdue' end as pmt_stt  from pawnoverduedetail pd 
   left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))  
   left join  (select pa1.*,shop.name shopname,shop.areaid from Pawn pa1 
   left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid  
   left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid   
where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt))  
and pa.STATUS not in (15,11,140) and pa.CategoryCode like N'%17%' and InterestMoney like N'%goc%' 
and pd.scheduletodate<=@schedule_to and pd.scheduletodate>=@schedule_from) tab
  left join (select pawnid pawn_wid,CodeNo,Body,
  case when [CommentType] = 1 then 'KH KNM'
        when [CommentType] = 2 then N'SĐT Không ll được'
        when [CommentType] = 3 then N'KH hẹn chuyển khoản'
        when [CommentType] = 4 then N'KH hẹn thanh toán'
        when [CommentType] = 5 then N'Người nghe báo sai số'
        when [CommentType] = 6 then N'Kh chặn số'
        when [CommentType] = 7 then N'KH báo thanh lý'
        when [CommentType] = 9 then N'Khác'
        when [CommentType] = 8 then 'Admin'
        else N'Khác' end CommentType,cmtdate,CreatedDate cmt_dt from CommentContract 
  where PGD is not null and cmtdate>= dateadd(day,-6,@schedule_from) and cmtdate<=@schedule_to)cmt 
  on cmt.pawn_wid=tab.pawnid and cmt.cmtdate>= dateadd(day,-6,tab.scheduletodate) and cmt.cmtdate<= tab.ScheduleToDate
 where dpd_yesterday<=5 """
    remind_stats = pd.read_sql_query(remind_str,db)
    remind_stats = remind_stats.merge(write_off_list, left_on='codeno', right_on='pawn_code',how = 'left')
    remind_stats = remind_stats[remind_stats['pawn_code'].isnull()==True]
    remind_stats = remind_stats.merge(shop_data, left_on='shopname', right_on = 'Tên PGD', how = 'left')
    remind_stats['khu_vuc'] = np.where(remind_stats['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(remind_stats['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((remind_stats['Tỉnh/ Thành phố'] != 'Hà Nội') & (remind_stats['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((remind_stats['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (remind_stats['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    return remind_stats

In [85]:
#  car title
def remind_rate_cartitle(fromdate, todate):
    remind_cartitle = """Declare @schedule_from date ='"""+fromdate+"""'
                Declare @schedule_to date ='"""+todate+"""'
                select  tab.*,cmt.*,
DATEDIFF(day,cmt.cmtdate,ScheduleToDate) day_cmt_before_schedule,
case when cmtdate is null then N'1.no_cmt'
when DATEDIFF(day,cmt.cmtdate,ScheduleToDate)=0 then N'3.on_due_date'
when DATEDIFF(day,cmt.cmtdate,ScheduleToDate)<>0 then N'2.early_remind' end as cmt_type,
case when PackageCode like '%NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
from ( select  pd.pawnid,po.tuoino dpd_yesterday,
pa.PackageCode,pa.shopname, pa.shopcode,
areaid,pd.scheduletodate,pd.paydate, pd.acc_prin_payment,
pa.status,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,   
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=acc_sum) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt, pd.type_cartitle
from (select 'monthly_paid_origin' type_cartitle, podd.[PawnID],podd.[ScheduleToDate] ,podd.[ScheduleFromDate]
      ,podd.[acc_prin_payment],podd.[paydate] ,podd.[InterestMoney], podd.acc_sum from [dwh].[dbo].[PawnOverDuedetail] podd
	  left join pawn pa on pa.pawnid = podd.pawnid
	  where InterestMoney like N'%goc%' and podd.scheduletodate<=@schedule_to and podd.scheduletodate>=@schedule_from
	  and pa.categorycode like '%15%'
	  union all
	  (select 'eop_paid_origin' type_cartitle, [PawnID]
      ,[ScheduleToDate]
      ,[ScheduleFromDate]
      ,[acc_prin_payment]
      ,[paydate]
      ,[MoneyInterest] as [InterestMoney],
	  acc_sum
  FROM [dwh].[dbo].[pawnCarOverDue]
  where [MoneyInterest] like '%lai%' and scheduletodate<=@schedule_to and scheduletodate>=@schedule_from)) pd 
   left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))  
   left join  (select pa1.*,shop.name shopname,shop.areaid from Pawn pa1 
   left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid  
   left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid   
where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt))  
and pa.STATUS not in (15,11,140) ) tab
  left join (select pawnid pawn_wid,CodeNo,Body,case when [CommentType] = 1 then 'KH KNM'
        when [CommentType] = 2 then N'SĐT Không ll được'
        when [CommentType] = 3 then N'KH hẹn chuyển khoản'
        when [CommentType] = 4 then N'KH hẹn thanh toán'
        when [CommentType] = 5 then N'Người nghe báo sai số'
        when [CommentType] = 6 then N'Kh chặn số'
        when [CommentType] = 7 then N'KH báo thanh lý'
        when [CommentType] = 9 then N'Khác'
        when [CommentType] = 8 then 'Admin'
        else N'Khác' end  CommentType,cmtdate,CreatedDate cmt_dt from CommentContract 
  where PGD is not null and cmtdate>= dateadd(day,-6,@schedule_from) and cmtdate<=@schedule_to)cmt 
  on cmt.pawn_wid=tab.pawnid and cmt.cmtdate>= dateadd(day,-6,tab.scheduletodate) and cmt.cmtdate<= tab.ScheduleToDate
 where dpd_yesterday<=5 
 order by tab.PawnID"""
    remind_stats = pd.read_sql_query(remind_cartitle,db)
    remind_stats = remind_stats.merge(write_off_list, left_on='codeno', right_on='pawn_code',how = 'left')
    remind_stats = remind_stats[remind_stats['pawn_code'].isnull()==True]
    remind_stats = remind_stats.merge(shop_data, left_on='shopname', right_on = 'Tên PGD', how = 'left')
    remind_stats['khu_vuc'] = np.where(remind_stats['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(remind_stats['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((remind_stats['Tỉnh/ Thành phố'] != 'Hà Nội') & (remind_stats['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((remind_stats['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (remind_stats['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    return remind_stats

In [86]:
#  car title
def remind_rate_cartitle(fromdate, todate):
    remind_cartitle = """Declare @schedule_from date ='"""+fromdate+"""'
                Declare @schedule_to date ='"""+todate+"""'
                select  tab.*,cmt.*,
DATEDIFF(day,cmt.cmtdate,ScheduleToDate) day_cmt_before_schedule,
case when cmtdate is null then N'1.no_cmt'
when DATEDIFF(day,cmt.cmtdate,ScheduleToDate)=0 then N'3.on_due_date'
when DATEDIFF(day,cmt.cmtdate,ScheduleToDate)<>0 then N'2.early_remind' end as cmt_type,
case when PackageCode like '%NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type
from ( select  pd.pawnid,po.tuoino dpd_yesterday,
pa.PackageCode,pa.shopname, pa.shopcode,
areaid,pd.scheduletodate,pd.paydate, pd.acc_prin_payment,
pa.status,pa.FromDate,pa.ToDate,pa.closedate,liquid_dt,   
case  when (pd.paydate<=ScheduleToDate  and acc_prin_payment>=acc_sum) or CloseDate<=ScheduleToDate 
then N'ontime' else N'overdue' end as pmt_stt, pd.type_cartitle
from (select 'monthly_paid_origin' type_cartitle, podd.[PawnID],podd.[ScheduleToDate] ,podd.[ScheduleFromDate]
      ,podd.[acc_prin_payment],podd.[paydate] ,podd.[InterestMoney], podd.acc_sum from [dwh].[dbo].[PawnOverDuedetail] podd
	  left join pawn pa on pa.pawnid = podd.pawnid
	  where InterestMoney like N'%goc%' and podd.scheduletodate<=@schedule_to and podd.scheduletodate>=@schedule_from
	  and pa.categorycode like '%15%'
	  union all
	  (select 'eop_paid_origin' type_cartitle, [PawnID]
      ,[ScheduleToDate]
      ,[ScheduleFromDate]
      ,[acc_prin_payment]
      ,[paydate]
      ,[MoneyInterest] as [InterestMoney],
	  acc_sum
  FROM [dwh].[dbo].[pawnCarOverDue]
  where [MoneyInterest] like '%lai%' and scheduletodate<=@schedule_to and scheduletodate>=@schedule_from)) pd 
   left join pawnoverdue po on (po.pawnid=pd.pawnid and po.created=dateadd(day,-1,pd.scheduletodate))  
   left join  (select pa1.*,shop.name shopname,shop.areaid from Pawn pa1 
   left join ShopDetail shop on pa1.shopcode=shop.code )pa on pa.pawnid=pd.pawnid  
   left join w_pawn_liquid_sale_f li on li.pawn_wid=pd.pawnid   
where ((closedate is null and liquid_dt is null) or(ScheduleToDate<=CloseDate) or (scheduletodate<=liquid_dt))  
and pa.STATUS not in (15,11,140) ) tab
  left join (select pawnid pawn_wid,CodeNo,Body,case when [CommentType] = 1 then 'KH KNM'
        when [CommentType] = 2 then N'SĐT Không ll được'
        when [CommentType] = 3 then N'KH hẹn chuyển khoản'
        when [CommentType] = 4 then N'KH hẹn thanh toán'
        when [CommentType] = 5 then N'Người nghe báo sai số'
        when [CommentType] = 6 then N'Kh chặn số'
        when [CommentType] = 7 then N'KH báo thanh lý'
        when [CommentType] = 9 then N'Khác'
        when [CommentType] = 8 then 'Admin'
        else N'Khác' end  CommentType,cmtdate,CreatedDate cmt_dt from CommentContract 
  where PGD is not null and cmtdate>= dateadd(day,-6,@schedule_from) and cmtdate<=@schedule_to)cmt 
  on cmt.pawn_wid=tab.pawnid and cmt.cmtdate>= dateadd(day,-6,tab.scheduletodate) and cmt.cmtdate<= tab.ScheduleToDate
 where dpd_yesterday<=5 
 order by tab.PawnID"""
    remind_stats = pd.read_sql_query(remind_cartitle,db)
    remind_stats = remind_stats.merge(write_off_list, left_on='codeno', right_on='pawn_code',how = 'left')
    remind_stats = remind_stats[remind_stats['pawn_code'].isnull()==True]
    remind_stats = remind_stats.merge(shop_data, left_on='shopname', right_on = 'Tên PGD', how = 'left')
    remind_stats['khu_vuc'] = np.where(remind_stats['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(remind_stats['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((remind_stats['Tỉnh/ Thành phố'] != 'Hà Nội') & (remind_stats['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((remind_stats['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (remind_stats['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
    return remind_stats

MOTOR TITLE

In [89]:
remind_stats_motor_title = remind_rate(date_list[0][0],date_list[0][1])

In [90]:
remind_stats_motor_title_last_month = remind_rate(date_list[1][0],date_list[1][1])

In [91]:
#  categroy, area
need_remind_motor_title_area = remind_stats_motor_title.groupby('khu_vuc').agg({'pawnid':'nunique'}).T.reset_index()
need_remind_motor_title_area_last_month = remind_stats_motor_title_last_month.groupby('khu_vuc').agg({'pawnid':'nunique'}).T.reset_index()

need_remind_motor_title_area = need_remind_motor_title_area[['index', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
need_remind_motor_title_area_last_month = need_remind_motor_title_area_last_month[['index', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]

need_remind_motor_title_area.columns = ['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']
need_remind_motor_title_area_last_month.columns = ['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']

In [92]:
remind_motor_title_area =remind_stats_motor_title.pivot_table(columns= 'khu_vuc', index='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_area = remind_motor_title_area.fillna(0)
remind_motor_title_area = remind_motor_title_area[['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
remind_motor_title_area = remind_motor_title_area.append(need_remind_motor_title_area)

remind_motor_title_area_last_month = remind_stats_motor_title_last_month.pivot_table(columns='khu_vuc', index='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_area_last_month = remind_motor_title_area_last_month.fillna(0)
remind_motor_title_area_last_month = remind_motor_title_area_last_month[['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
remind_motor_title_area_last_month = remind_motor_title_area_last_month.append(need_remind_motor_title_area_last_month)

In [93]:
# shop, motor title
need_remind_motor_title_shop = remind_stats_motor_title.groupby('shopname').agg({'pawnid':'nunique'}).reset_index()
need_remind_motor_title_shop_last_month = remind_stats_motor_title_last_month.groupby('shopname').agg({'pawnid':'nunique'}).reset_index()

remind_motor_title_shop = remind_stats_motor_title.pivot_table(index= 'shopname', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_shop = remind_motor_title_shop.fillna(0)
remind_motor_title_shop = remind_motor_title_shop.merge(need_remind_motor_title_shop, on = 'shopname', how = 'left')

remind_motor_title_shop_last_month = remind_stats_motor_title_last_month.pivot_table(index= 'shopname', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_shop_last_month = remind_motor_title_shop_last_month.fillna(0)
remind_motor_title_shop_last_month = remind_motor_title_shop_last_month.merge(need_remind_motor_title_shop_last_month, on = 'shopname', how = 'left')

remind_motor_title_shop['early_remind_rate'] = remind_motor_title_shop['2.early_remind']/remind_motor_title_shop['pawnid']
remind_motor_title_shop['ondue_remind_rate'] = remind_motor_title_shop['3.on_due_date']/remind_motor_title_shop['pawnid']

remind_motor_title_shop_last_month['early_remind_rate'] = remind_motor_title_shop_last_month['2.early_remind']/remind_motor_title_shop_last_month['pawnid']
remind_motor_title_shop_last_month['ondue_remind_rate'] = remind_motor_title_shop_last_month['3.on_due_date']/remind_motor_title_shop_last_month['pawnid']

In [94]:
# asm
need_remind_motor_title_asm = remind_stats_motor_title.groupby('QLKV').agg({'pawnid':'nunique'}).reset_index()
need_remind_motor_title_asm_last_month = remind_stats_motor_title_last_month.groupby('QLKV').agg({'pawnid':'nunique'}).reset_index()

remind_motor_title_asm = remind_stats_motor_title.pivot_table(index= 'QLKV', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_asm = remind_motor_title_asm.fillna(0)
remind_motor_title_asm = remind_motor_title_asm.merge(need_remind_motor_title_asm, on = 'QLKV', how = 'left')

remind_motor_title_asm_last_month = remind_stats_motor_title_last_month.pivot_table(index= 'QLKV', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_asm_last_month = remind_motor_title_asm_last_month.fillna(0)
remind_motor_title_asm_last_month = remind_motor_title_asm_last_month.merge(need_remind_motor_title_asm_last_month, on = 'QLKV', how = 'left')

remind_motor_title_asm['early_remind_rate'] = remind_motor_title_asm['2.early_remind']/remind_motor_title_asm['pawnid']
remind_motor_title_asm['ondue_remind_rate'] = remind_motor_title_asm['3.on_due_date']/remind_motor_title_asm['pawnid']
remind_motor_title_asm_last_month['early_remind_rate'] = remind_motor_title_asm_last_month['2.early_remind']/remind_motor_title_asm_last_month['pawnid']
remind_motor_title_asm_last_month['ondue_remind_rate'] = remind_motor_title_asm_last_month['3.on_due_date']/remind_motor_title_asm_last_month['pawnid']

In [95]:
# schedule
need_remind_motor_title_schedule = remind_stats_motor_title.groupby('scheduletodate').agg({'pawnid':'nunique'}).reset_index()
need_remind_motor_title_schedule_last_month = remind_stats_motor_title_last_month.groupby('scheduletodate').agg({'pawnid':'nunique'}).reset_index()
remind_motor_title_schedule = remind_stats_motor_title.pivot_table(index= 'scheduletodate', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_schedule = remind_motor_title_schedule.fillna(0)
remind_motor_title_schedule = remind_motor_title_schedule.merge(need_remind_motor_title_schedule, on = 'scheduletodate', how = 'left')

remind_motor_title_schedule_last_month = remind_stats_motor_title_last_month.pivot_table(index= 'scheduletodate', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_schedule_last_month = remind_motor_title_schedule_last_month.fillna(0)
remind_motor_title_schedule_last_month = remind_motor_title_schedule_last_month.merge(need_remind_motor_title_schedule_last_month, on = 'scheduletodate', how = 'left')

remind_motor_title_schedule['early_remind_rate'] = remind_motor_title_schedule['2.early_remind']/remind_motor_title_schedule['pawnid']
remind_motor_title_schedule['ondue_remind_rate'] = remind_motor_title_schedule['3.on_due_date']/remind_motor_title_schedule['pawnid']
remind_motor_title_schedule_last_month['early_remind_rate'] = remind_motor_title_schedule_last_month['2.early_remind']/remind_motor_title_schedule_last_month['pawnid']
remind_motor_title_schedule_last_month['ondue_remind_rate'] = remind_motor_title_schedule_last_month['3.on_due_date']/remind_motor_title_schedule_last_month['pawnid']

CAR TITLE

In [96]:
remind_stats_car_title = remind_rate_cartitle(date_list[0][0],date_list[0][1])

In [97]:
remind_stats_car_title_last_month = remind_rate_cartitle(date_list[1][0],date_list[1][1])

In [98]:
#  categroy, area
need_remind_car_title_area = remind_stats_car_title.groupby('khu_vuc').agg({'pawnid':'nunique'}).T.reset_index()
need_remind_car_title_area_last_month = remind_stats_car_title_last_month.groupby('khu_vuc').agg({'pawnid':'nunique'}).T.reset_index()

need_remind_car_title_area = need_remind_car_title_area[['index', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
need_remind_car_title_area_last_month = need_remind_car_title_area_last_month[['index', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]

need_remind_car_title_area.columns = ['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']
need_remind_car_title_area_last_month.columns = ['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']

In [99]:
remind_car_title_area =remind_stats_car_title.pivot_table(columns= 'khu_vuc', index='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_area = remind_car_title_area.fillna(0)
remind_car_title_area = remind_car_title_area[['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
remind_car_title_area = remind_car_title_area.append(need_remind_car_title_area)

remind_car_title_area_last_month = remind_stats_car_title_last_month.pivot_table(columns='khu_vuc', index='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_area_last_month = remind_car_title_area_last_month.fillna(0)
remind_car_title_area_last_month = remind_car_title_area_last_month[['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
remind_car_title_area_last_month = remind_car_title_area_last_month.append(need_remind_car_title_area_last_month)

In [100]:
# shop, car title
need_remind_car_title_shop = remind_stats_car_title.groupby('shopname').agg({'pawnid':'nunique'}).reset_index()
need_remind_car_title_shop_last_month = remind_stats_car_title_last_month.groupby('shopname').agg({'pawnid':'nunique'}).reset_index()

remind_car_title_shop = remind_stats_car_title.pivot_table(index= 'shopname', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_shop = remind_car_title_shop.fillna(0)
remind_car_title_shop = remind_car_title_shop.merge(need_remind_car_title_shop, on = 'shopname', how = 'left')

remind_car_title_shop_last_month = remind_stats_car_title_last_month.pivot_table(index= 'shopname', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_shop_last_month = remind_car_title_shop_last_month.fillna(0)
remind_car_title_shop_last_month = remind_car_title_shop_last_month.merge(need_remind_car_title_shop_last_month, on = 'shopname', how = 'left')

remind_car_title_shop['early_remind_rate'] = remind_car_title_shop['2.early_remind']/remind_car_title_shop['pawnid']
remind_car_title_shop['ondue_remind_rate'] = remind_car_title_shop['3.on_due_date']/remind_car_title_shop['pawnid']

remind_car_title_shop_last_month['early_remind_rate'] = remind_car_title_shop_last_month['2.early_remind']/remind_car_title_shop_last_month['pawnid']
remind_car_title_shop_last_month['ondue_remind_rate'] = remind_car_title_shop_last_month['3.on_due_date']/remind_car_title_shop_last_month['pawnid']

In [101]:
# asm
need_remind_car_title_asm = remind_stats_car_title.groupby('QLKV').agg({'pawnid':'nunique'}).reset_index()
need_remind_car_title_asm_last_month = remind_stats_car_title_last_month.groupby('QLKV').agg({'pawnid':'nunique'}).reset_index()

remind_car_title_asm = remind_stats_car_title.pivot_table(index= 'QLKV', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_asm = remind_car_title_asm.fillna(0)
remind_car_title_asm = remind_car_title_asm.merge(need_remind_car_title_asm, on = 'QLKV', how = 'left')

remind_car_title_asm_last_month = remind_stats_car_title_last_month.pivot_table(index= 'QLKV', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_asm_last_month = remind_car_title_asm_last_month.fillna(0)
remind_car_title_asm_last_month = remind_car_title_asm_last_month.merge(need_remind_car_title_asm_last_month, on = 'QLKV', how = 'left')

remind_car_title_asm['early_remind_rate'] = remind_car_title_asm['2.early_remind']/remind_car_title_asm['pawnid']
remind_car_title_asm['ondue_remind_rate'] = remind_car_title_asm['3.on_due_date']/remind_car_title_asm['pawnid']
remind_car_title_asm_last_month['early_remind_rate'] = remind_car_title_asm_last_month['2.early_remind']/remind_car_title_asm_last_month['pawnid']
remind_car_title_asm_last_month['ondue_remind_rate'] = remind_car_title_asm_last_month['3.on_due_date']/remind_car_title_asm_last_month['pawnid']

In [102]:
# schedule
need_remind_car_title_schedule = remind_stats_car_title.groupby('scheduletodate').agg({'pawnid':'nunique'}).reset_index()
need_remind_car_title_schedule_last_month = remind_stats_car_title_last_month.groupby('scheduletodate').agg({'pawnid':'nunique'}).reset_index()
remind_car_title_schedule = remind_stats_car_title.pivot_table(index= 'scheduletodate', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_schedule = remind_car_title_schedule.fillna(0)
remind_car_title_schedule = remind_car_title_schedule.merge(need_remind_car_title_schedule, on = 'scheduletodate', how = 'left')

remind_car_title_schedule_last_month = remind_stats_car_title_last_month.pivot_table(index= 'scheduletodate', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_schedule_last_month = remind_car_title_schedule_last_month.fillna(0)
remind_car_title_schedule_last_month = remind_car_title_schedule_last_month.merge(need_remind_car_title_schedule_last_month, on = 'scheduletodate', how = 'left')

remind_car_title_schedule['early_remind_rate'] = remind_car_title_schedule['2.early_remind']/remind_car_title_schedule['pawnid']
remind_car_title_schedule['ondue_remind_rate'] = remind_car_title_schedule['3.on_due_date']/remind_car_title_schedule['pawnid']
remind_car_title_schedule_last_month['early_remind_rate'] = remind_car_title_schedule_last_month['2.early_remind']/remind_car_title_schedule_last_month['pawnid']
remind_car_title_schedule_last_month['ondue_remind_rate'] = remind_car_title_schedule_last_month['3.on_due_date']/remind_car_title_schedule_last_month['pawnid']

REMIND BY PROVINCE

In [103]:
remind_stats_motor_title.columns

Index(['pawnid', 'dpd_yesterday', 'packagecode', 'shopname', 'shopcode',
       'areaid', 'scheduletodate', 'principalmoney', 'acc_sum', 'paydate',
       'acc_prin_payment', 'paidmoney', 'status', 'fromdate', 'todate',
       'closedate', 'liquid_dt', 'pmt_stt', 'pawn_wid', 'codeno', 'body',
       'commenttype', 'cmtdate', 'cmt_dt', 'day_cmt_before_schedule',
       'cmt_type', 'package_type', 'write_off_period', 'pawn_code', 'amount',
       'Tên PGD', 'Mã PGD', 'Phường/Xã', 'Quận/Huyện', 'Tỉnh/ Thành phố',
       'Vĩ độ', 'Kinh độ', 'Shop ID', 'Vùng/Miền', 'Thời gian KT', 'Vùng',
       'code', 'name', 'GĐM', 'TPK', 'QLKV', 'khu_vuc'],
      dtype='object')

In [104]:
# Motor title
#  categroy, province
need_remind_motor_title_province = remind_stats_motor_title.groupby('Tỉnh/ Thành phố').agg({'pawnid':'nunique'}).reset_index()
need_remind_motor_title_province_last_month = remind_stats_motor_title_last_month.groupby('Tỉnh/ Thành phố').agg({'pawnid':'nunique'}).reset_index()

In [105]:
remind_motor_title_province =remind_stats_motor_title.pivot_table(index= 'Tỉnh/ Thành phố', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_province = remind_motor_title_province.fillna(0)
# remind_car_title_area = remind_car_title_area[['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
# remind_car_title_area = remind_car_title_area.append(need_remind_car_title_area)

remind_motor_title_province_last_month = remind_stats_motor_title_last_month.pivot_table(index='Tỉnh/ Thành phố', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_motor_title_province_last_month = remind_motor_title_province_last_month.fillna(0)
# remind_car_title_area_last_month = remind_car_title_area_last_month[['cmt_type', 'Hà Nội', 'Tỉnh MB','TP. Hồ Chí Minh','Tỉnh MN','Miền Trung']]
# remind_car_title_area_last_month = remind_car_title_area_last_month.append(need_remind_car_title_area_last_month)

In [106]:
remind_motor_title_province = remind_motor_title_province.merge(need_remind_motor_title_province, on = 'Tỉnh/ Thành phố', how = 'left')

In [107]:
remind_motor_title_province['early_remind_rate'] = remind_motor_title_province['2.early_remind']/remind_motor_title_province['pawnid']

In [108]:
remind_motor_title_province_last_month = remind_motor_title_province_last_month.merge(need_remind_motor_title_province_last_month, on = 'Tỉnh/ Thành phố', how = 'left')

In [109]:
remind_motor_title_province_last_month['early_remind_rate'] = remind_motor_title_province_last_month['2.early_remind']/remind_motor_title_province_last_month['pawnid']

In [110]:
need_remind_car_title_province = remind_stats_car_title.groupby('Tỉnh/ Thành phố').agg({'pawnid':'nunique'}).reset_index()
need_remind_car_title_province_last_month = remind_stats_car_title_last_month.groupby('Tỉnh/ Thành phố').agg({'pawnid':'nunique'}).reset_index()

In [111]:
remind_car_title_province =remind_stats_car_title.pivot_table(index= 'Tỉnh/ Thành phố', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_province = remind_car_title_province.fillna(0)

remind_car_title_province_last_month = remind_stats_car_title_last_month.pivot_table(index='Tỉnh/ Thành phố', columns='cmt_type',values='pawnid', aggfunc='nunique').reset_index()
remind_car_title_province_last_month = remind_car_title_province_last_month.fillna(0)

In [112]:
remind_car_title_province = remind_car_title_province.merge(need_remind_car_title_province, on = 'Tỉnh/ Thành phố', how = 'left')

In [113]:
remind_car_title_province_last_month = remind_car_title_province_last_month.merge(need_remind_car_title_province_last_month, on = 'Tỉnh/ Thành phố', how = 'left')

In [114]:
remind_car_title_province['early_remind_rate'] = remind_car_title_province['2.early_remind']/remind_car_title_province['pawnid']

In [115]:
remind_car_title_province_last_month['early_remind_rate'] = remind_car_title_province_last_month['2.early_remind']/remind_car_title_province_last_month['pawnid']

ADMIN REMIND

In [116]:
# def no_admin_remind(fromdate, todate):
#     admin_type1_str = """ Declare @fromdate date = '"""+fromdate+"""'
# 	Declare @todate date = '"""+todate+"""'
#    select sdl.*,pa.codeno, cc.body, cc.cmtdate,pa.status,pa.closedate, 
#    case when cmtdate is not null and cmtdate>dateadd(d, -3, sdl.scheduletodate)
#    and cmtdate<dateadd(d, 0, sdl.scheduletodate) then 't' else 'f' end t_f, pp.AdvanceMoney, pp.PayDate payment_date 
#    from (
#    select pawnid,ScheduleToDate,InterestMoney, acc_sum, acc_sum_l1, PayDate
#    from [dwh].[dbo].[pawnCarOverDue]
#    where scheduletodate >= @fromdate
#    and scheduletodate <= @todate
#     and [MoneyInterest] like '%lai%') sdl
# 	left join [dwh].[dbo].pawnoverdue pod on pod.PawnID = sdl.PawnID
#    left join (SELECT [PGD]
#        ,[PawnID]
#        ,[CodeNo]
#         ,[Body]
#         ,[CommentType] 
#        ,[CreatedDate]
#         ,[cmtdate]
#    FROM [dwh].[dbo].[CommentContract]
#    where Nhóm like '%admin%' 
#     and cmtdate>=DATEADD(d, -15, @fromdate)) cc on cc.pawnid = sdl.pawnid 
#     left join [dwh].[dbo].pawn pa on pa.pawnid = sdl.pawnid
# 	left join [dwh].[dbo].[PawnPayment] pp on (pp.PawnID = sdl.PawnID and pp.ToDate = sdl.ScheduleToDate)
# 	where pod.CREATED = @todate
# 	and pod.TuoiNo <=10
# 	  and pa.status in (-1,1,14)
# 	  and pp.PreAdvanceMoney <= 0 
# 	  and pp.PayDate >= DATEADD(d, -2, sdl.ScheduleToDate)
# 	  and (pa.CloseDate is null 
# 	  or pa.CloseDate<@fromdate)"""
    
#     admin_type2_str = """ Declare @fromdate date = '"""+fromdate+"""'
# 	Declare @todate date = '"""+todate+"""'
#    select sdl.*,pa.codeno, cc.body, cc.cmtdate,pa.status,pa.closedate, 
#    case when cmtdate is not null and cmtdate>dateadd(d, -3, sdl.scheduletodate)
#    and cmtdate<dateadd(d, 0, sdl.scheduletodate) then 't' else 'f' end t_f, pp.AdvanceMoney, pp.PayDate payment_date 
#    from (select pawnid,ScheduleToDate, PrincipalMoney InterestMoney, acc_sum, acc_sum_l1, PayDate from [dwh].[dbo].[PawnOverDuedetail]
#    where scheduletodate >= @fromdate
#    and scheduletodate <= @todate
#     and [InterestMoney] like '%goc%'
#    ) sdl
# 	left join [dwh].[dbo].pawnoverdue pod on pod.PawnID = sdl.PawnID
#    left join (SELECT [PGD]
#        ,[PawnID]
#        ,[CodeNo]
#         ,[Body]
#         ,[CommentType]
#        ,[CreatedDate]
#         ,[cmtdate]
#    FROM [dwh].[dbo].[CommentContract]
#    where Nhóm like '%admin%' 
#     and cmtdate>=DATEADD(d, -2, @fromdate)) cc on cc.pawnid = sdl.pawnid 
#     left join [dwh].[dbo].pawn pa on pa.pawnid = sdl.pawnid
# 	left join [dwh].[dbo].[PawnPayment] pp on (pp.PawnID = sdl.PawnID and pp.ToDate = sdl.ScheduleToDate)
# 	where pod.CREATED = @todate
# 	and pod.TuoiNo <=10
# 	  and pa.status in (-1,1,14)
# 	  and pp.PreAdvanceMoney <= 0 
# 	  and pp.PayDate >= DATEADD(d, -2, sdl.ScheduleToDate)
# 	  and (pa.CloseDate is null 
# 	  or pa.CloseDate<@fromdate)"""
#     admin_type1 = pd.read_sql_query(admin_type1_str, db)
#     admin_type2 = pd.read_sql_query(admin_type2_str, db)
#     admin_all = admin_type1.append(admin_type2)
#     return admin_all

In [117]:
# admin_this_month = no_admin_remind(date_list[0][0],date_list[0][1])

In [118]:
# admin_this_month.columns

In [119]:
# admind_no_remind = admin_this_month.pivot_table(index = ['scheduletodate','codeno'], columns='t_f', values='pawnid', aggfunc= 'count').reset_index()

In [120]:
# admind_no_remind = admind_no_remind[admind_no_remind['t'].isnull() == True]

In [121]:
# admind_no_remind

In [122]:
# admind_remind_summary = admin_this_month.pivot_table(index = 'scheduletodate', columns='admin_remind_yes_no', values='pawnid', aggfunc='nunique').reset_index()

In [123]:
# admind_remind_summary = admind_remind_summary.fillna(0)

In [124]:
# admind_remind_summary['all'] = admind_remind_summary['yes']+admind_remind_summary['no']
# admind_remind_summary['no_admin_remind_rate'] = admind_remind_summary['no']/admind_remind_summary['all']

In [125]:
# no_admin_remind_list_pawn = admin_this_month[admin_this_month['body'].isnull()== True]

In [126]:
# no_admin_remind_list_pawn

In [127]:
# # Comment on Pos
# # Comment group
# cmt_group_motor_title= remind_stats_motor_title.pivot_table(columns='commenttype', index = 'cmt_type', values='pawnid', aggfunc='nunique').reset_index()

# cmt_group_car_title= remind_stats_car_title.pivot_table(columns='commenttype', index = 'cmt_type', values='pawnid', aggfunc='nunique').reset_index()


In [128]:
# cmt_group_car_title.columns

In [129]:
# cmt_group_motor_title.columns = ['cmt_type','KH KNM','SĐT Không ll được','KH hẹn chuyển khoản','KH hẹn thanh toán','Người nghe báo sai số','Kh chặn số','KH báo thanh lý','Admin','Khác']
# cmt_group_car_title.columns = ['cmt_type','KH KNM','SĐT Không ll được','KH hẹn chuyển khoản','KH báo thanh lý','KH hẹn thanh toán','Admin','Khác']

CLOSED PAWN

In [130]:
# #  Active pawn at the beginning of this month
# def closed_pawn_type_1(fromdate, todate):
#     closed_type_1_str = """Declare @start_date date ='"""+fromdate+"""'
#     Declare @end_date date ='"""+todate+"""'
# select pod.categoryname,pod.shopcode,pod.ShopName shopname,pod.contractcode, pod.pawnid pod_pawn,(pa2.LoanMoneyOrg - pa2.[InsurranceMoney])/1000000 all_cal_interest,
# cast(left(pod.Interest,4) as int) interest, pa.net_disbur/1000000 closed_cal_interest, pa.*,
# case  when pa.Pawnid is null then 0 else 1 end real_closed_pawn
# from PawnOverdue pod
# left join (select a.*, case when a.closedate < a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '1. early closed'
# when  a.closedate = a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '2. ondue closed'
# when a.closedate > a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '3. overdue_closed'
#  when a.closedate=a.following_fromdate then '4.close to open new'
#  else '5.just closed' end "closed_type", 
#  case when format(fromdate, 'yyyyMM') = format(closedate, 'yyyyMM') then  'oc same month'
#  else 'oc diff month' end oc_month_compare,
#  case when CategoryCode like '%17%' then
# case when (a.foll_loan_org - a.foll_pawn_insurance) <= 3000000 then '0. <=3tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 10000000 then '1. 3-10tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 15000000 then '2. 10-15tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 25000000 then '3. 15-25tr'
# else '4. >25tr' end 
#   when CategoryCode like '%15%' then
# case when (a.foll_loan_org - a.foll_pawn_insurance) <= 50000000 then '0. <=50tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 100000000 then '2. 100-200tr'
# when a.foll_pawn_package is not null and  (a.foll_loan_org - a.foll_pawn_insurance) > 200000000 then N'4. >200tr'
# when a.foll_pawn_package is null and  (a.foll_loan_org - a.foll_pawn_insurance) <= 300000000 then '3. 200-300tr'
# else N'4. >300tr' end 
#  end "foll_package_money_range", 
#  case when a.foll_pawn_frequency<= 30 then N'1. Kỳ hạn 1 tháng'
#   when a.foll_pawn_frequency <= 90 then N'2. Kỳ hạn 3 tháng'
#   when a.foll_pawn_frequency  <= 180 then N'3. Kỳ hạn 6 tháng'
#   when a.foll_pawn_frequency  <= 270 then N'4. Kỳ hạn 9 tháng'
#   when a.foll_pawn_frequency  <= 360 then N'5. Kỳ hạn 12 tháng'
#   else N'6. >12 tháng' end foll_package_time_range, 
#   case when a.foll_pawn_package like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as foll_package_type
# from (
# select [CategoryCode], pawnid, fromdate, todate, closedate,  Lead (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_pawn_id,
#  Lead (fromdate) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_fromdate,
#  case when CategoryCode like '%17%' then
# case when (LoanMoneyOrg - [InsurranceMoney]) <= 3000000 then '0. <=3tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 10000000 then '1. 3-10tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 15000000 then '2. 10-15tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 25000000 then '3. 15-25tr'
# else '4. >25tr' end
#   when CategoryCode like '%15%' then
# case when (LoanMoneyOrg - [InsurranceMoney]) <= 50000000 then '0. <=50tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 100000000 then '2. 100-200tr'
# when PackageCode is not null and  (LoanMoneyOrg - [InsurranceMoney]) > 200000000 then N'4. >200tr'
# when PackageCode is null and  (LoanMoneyOrg - [InsurranceMoney]) <= 300000000 then '3. 200-300tr'
# else N'4. >300tr' end 
#  end "package_money_range", case when frequency <= 30 then N'1. Kỳ hạn 1 tháng'
#   when frequency <= 90 then N'2. Kỳ hạn 3 tháng'
#   when frequency <= 180 then N'3. Kỳ hạn 6 tháng'
#   when frequency <= 270 then N'4. Kỳ hạn 9 tháng'
#   when frequency <= 360 then N'5. Kỳ hạn 12 tháng'
#   else N'6. Ngoại lệ' end package_time_range,
#   case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type,
#   Lead(CodeNo) over (PARTITION BY customerid, CategoryCode ORDER BY fromdate asc) as foll_pawn, 
#   Lead(Pawnid) over (PARTITION BY customerid, CategoryCode ORDER BY fromdate asc) as foll_pawn_id,
# Lead(InsurranceMoney) over (PARTITION BY customerid,CategoryCode  order by fromdate) as foll_pawn_insurance,
# Lead(LoanMoneyOrg) over (PARTITION BY customerid, CategoryCode order by fromdate) as foll_loan_org,
# Lead(PackageCode) over (PARTITION BY customerid, CategoryCode order by fromdate) foll_pawn_package,
# Lead(Frequency) over (PARTITION BY customerid,CategoryCode  order by fromdate) as foll_pawn_frequency,
# (LoanMoneyOrg - InsurranceMoney) net_disbur
#  from pawn
#  where closedate> @start_date
#  and closedate<=@end_date) a) pa on pa.pawnid = pod.PawnID
#  left join pawn pa2 on pa2.pawnid = pod.pawnid
#  where pod.CREATED = @start_date
#  and (pod.CategoryCode like '%15%'
#  or pod.CategoryCode like '%17%')
#  and pod.shopcode not like '%HS%' 
# and pod.shopcode not like '%TEST%'
# and pod.TuoiNo<90"""
#     closed_type_1 = pd.read_sql_query(closed_type_1_str,db)
#     closed_type_1 = closed_type_1.merge(write_off_list, left_on='contractcode', right_on='pawn_code',how = 'left')
#     closed_type_1 = closed_type_1[closed_type_1['pawn_code'].isnull()==True]
#     closed_type_1 = closed_type_1.merge(shop_data, left_on='shopname', right_on = 'Tên PGD', how = 'left')
#     closed_type_1['khu_vuc'] = np.where(closed_type_1['Tỉnh/ Thành phố'] == 'Hà Nội', 'Hà Nội', np.where(closed_type_1['Tỉnh/ Thành phố'] == 'TP. Hồ Chí Minh', 'TP. Hồ Chí Minh',np.where((closed_type_1['Tỉnh/ Thành phố'] != 'Hà Nội') & (closed_type_1['Vùng/Miền'] == 'Miền Bắc'), 'Tỉnh MB', np.where((closed_type_1['Tỉnh/ Thành phố'] != 'TP. Hồ Chí Minh') & (closed_type_1['Vùng/Miền'] == 'Miền Nam'), 'Tỉnh MN', 'Miền Trung'))))
#     closed_type_1['expected_collect'] = closed_type_1['interest']*closed_type_1['all_cal_interest']
#     closed_type_1['closed_amt'] = closed_type_1['interest']*closed_type_1['closed_cal_interest']
#     return closed_type_1

In [131]:
# closed_type_1 = closed_pawn_type_1(date_list[0][0], date_list[0][1])

In [132]:
# closed_ty1_cate = closed_type_1.groupby('categoryname').agg({'pod_pawn':'nunique','real_closed_pawn':'sum', 'expected_collect':'sum', 'closed_amt':'sum'}).reset_index()

In [133]:
# closed_ty1_openty = closed_type_1[closed_type_1['closed_type'].isnull() == False]

# closed_ty1_openty = closed_ty1_openty.pivot_table(columns = 'closed_type', index = 'categoryname', values='pawnid', aggfunc = 'nunique').reset_index()

# closed_ty1_overview = closed_ty1_cate.merge(closed_ty1_openty, on = 'categoryname', how = 'left')


In [134]:
# closed_ty1_overview

In [135]:
# closed_ty1_oc = closed_type_1[closed_type_1['pawnid'].isnull() == False]

# closed_ty1_oc = closed_ty1_oc.pivot_table(index='categoryname', columns='oc_month_compare', values='pawnid', aggfunc= 'nunique')

# closed_ty1_oc = closed_ty1_oc.reset_index()


In [136]:
# closed_type_1_lm = closed_pawn_type_1(date_list[1][0], date_list[1][1])

# closed_type_1_lm = closed_type_1_lm.fillna(0)

# closed_ty1_cate_lm = closed_type_1_lm.groupby('categoryname').agg({'pod_pawn':'nunique','real_closed_pawn':'sum', 'expected_collect':'sum', 'closed_amt':'sum'})

# closed_ty1_openty_lm = closed_type_1_lm[closed_type_1_lm['closed_type'].isnull() == False]

# closed_ty1_openty_lm = closed_ty1_openty_lm.pivot_table(columns = 'closed_type', index = 'categoryname', values='pawnid', aggfunc = 'nunique').reset_index()

# closed_ty1_overview_lm = closed_ty1_cate.merge(closed_ty1_openty_lm, on = 'categoryname', how = 'left')

# closed_ty1_oc_lm = closed_type_1_lm[closed_type_1_lm['pawnid'].isnull() == False]

# closed_ty1_oc_lm = closed_ty1_oc_lm.pivot_table(index='categoryname', columns='oc_month_compare', values='pawnid', aggfunc= 'nunique')

# closed_ty1_oc_lm = closed_ty1_oc_lm.reset_index()

In [137]:
# # Pawn open in this month:
# def closed_pawn_type_2(fromdate,todate):
#     closed_type_2_str = """Declare @start_date date ='"""+fromdate+"""'
# Declare @end_date date ='"""+todate+"""'
# select a.*,cd.name categoryname, case when closedate is null then '0.still_open'
# when closedate is not null and a.closedate < a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '1. early closed'
# when closedate is not null and a.closedate = a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '2. ondue closed'
# when closedate is not null and a.closedate > a.todate and (a.closedate<>a.following_fromdate or a.following_fromdate is null) then '3. overdue_closed'
#  when closedate is not null and a.closedate=a.following_fromdate then '4.close to open new'
#  else '4.just closed' end "closed_type", 
#  case when format(fromdate, 'yyyyMM') = format(closedate, 'yyyyMM') then  'oc same month'
#  else 'oc diff month' end oc_month_compare,
#  case when CategoryCode like '%17%' then
# case when (a.foll_loan_org - a.foll_pawn_insurance) <= 3000000 then '0. <=3tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 10000000 then '1. 3-10tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 15000000 then '2. 10-15tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 25000000 then '3. 15-25tr'
# else '4. >25tr' end 
#   when CategoryCode like '%15%' then
# case when (a.foll_loan_org - a.foll_pawn_insurance) <= 50000000 then '0. <=50tr'
# when (a.foll_loan_org - a.foll_pawn_insurance) <= 100000000 then '2. 100-200tr'
# when a.foll_pawn_package is not null and  (a.foll_loan_org - a.foll_pawn_insurance) > 200000000 then N'4. >200tr'
# when a.foll_pawn_package is null and  (a.foll_loan_org - a.foll_pawn_insurance) <= 300000000 then '3. 200-300tr'
# else N'4. >300tr' end 
#  end "foll_package_money_range", 
#  case when a.foll_pawn_frequency<= 30 then N'1. Kỳ hạn 1 tháng'
#   when a.foll_pawn_frequency <= 90 then N'2. Kỳ hạn 3 tháng'
#   when a.foll_pawn_frequency  <= 180 then N'3. Kỳ hạn 6 tháng'
#   when a.foll_pawn_frequency  <= 270 then N'4. Kỳ hạn 9 tháng'
#   when a.foll_pawn_frequency  <= 360 then N'5. Kỳ hạn 12 tháng'
#   else N'6. >12 tháng' end foll_package_time_range, 
#   case when a.foll_pawn_package like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as foll_package_type,
#   a.rate*1000*(a.LoanMoneyOrg - a.[InsurranceMoney]) expected_amt
# from (
# select shopcode, rate,[CategoryCode], pawnid, fromdate, todate, closedate,  LoanMoneyOrg,[InsurranceMoney],
# Lead (Pawnid) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_pawn_id,
#  Lead (fromdate) over (PARTITION BY customerid, [CategoryCode] ORDER BY fromdate asc) as following_fromdate,
#  case when CategoryCode like '%17%' then
# case when (LoanMoneyOrg - [InsurranceMoney]) <= 3000000 then '0. <=3tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 10000000 then '1. 3-10tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 15000000 then '2. 10-15tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 25000000 then '3. 15-25tr'
# else '4. >25tr' end
#   when CategoryCode like '%15%' then
# case when (LoanMoneyOrg - [InsurranceMoney]) <= 50000000 then '0. <=50tr'
# when (LoanMoneyOrg - [InsurranceMoney]) <= 100000000 then '2. 100-200tr'
# when PackageCode is not null and  (LoanMoneyOrg - [InsurranceMoney]) > 200000000 then N'4. >200tr'
# when PackageCode is null and  (LoanMoneyOrg - [InsurranceMoney]) <= 300000000 then '3. 200-300tr'
# else N'4. >300tr' end 
#  end "package_money_range", case when frequency <= 30 then N'1. Kỳ hạn 1 tháng'
#   when frequency <= 90 then N'2. Kỳ hạn 3 tháng'
#   when frequency <= 180 then N'3. Kỳ hạn 6 tháng'
#   when frequency <= 270 then N'4. Kỳ hạn 9 tháng'
#   when frequency <= 360 then N'5. Kỳ hạn 12 tháng'
#   else N'6. Ngoại lệ' end package_time_range,
#   case when PackageCode like '%VAY_NHANH%' then N'Vay nhanh' else N'Vay thường' end as package_type,
#   Lead(CodeNo) over (PARTITION BY customerid, CategoryCode ORDER BY fromdate asc) as foll_pawn, 
#   Lead(Pawnid) over (PARTITION BY customerid, CategoryCode ORDER BY fromdate asc) as foll_pawn_id,
# Lead(InsurranceMoney) over (PARTITION BY customerid,CategoryCode  order by fromdate) as foll_pawn_insurance,
# Lead(LoanMoneyOrg) over (PARTITION BY customerid, CategoryCode order by fromdate) as foll_loan_org,
# Lead(PackageCode) over (PARTITION BY customerid, CategoryCode order by fromdate) foll_pawn_package,
# Lead(Frequency) over (PARTITION BY customerid,CategoryCode  order by fromdate) as foll_pawn_frequency
#  from pawn
#  where 1=1
#  and fromdate> @start_date
#  and fromdate<=@end_date
#  and shopcode not like '%HS%' and 
# shopcode not like '%TEST%'
# and (categorycode like '%17%'or categorycode like '%15%') 
# ) a
#  left join W_CATEGORY_D cd on cd.CODE = a.CategoryCode"""
#     closed_type_2 = pd.read_sql_query(closed_type_2_str, db)
#     return closed_type_2

In [138]:
# closed_type_2 = closed_pawn_type_2(date_list[0][0], date_list[0][1])

# closed_ty2_overview = closed_type_2.pivot_table(index='categoryname', columns='closed_type', values='pawnid',aggfunc='nunique')
# closed_ty2_overview = closed_ty2_overview.reset_index()
# closed_ty2_overview_amt = closed_type_2.pivot_table(index='categoryname', columns='closed_type', values='expected_amt',aggfunc='sum')
# closed_ty2_oc = closed_type_2.pivot_table(index=['categoryname', 'closed_type'], columns='oc_month_compare', values='pawnid', aggfunc='nunique')
# closed_ty2_oc_amt = closed_type_2.pivot_table(indexA=['categoryname', 'closed_type'], columns='oc_month_compare', values='expected_amt', aggfunc='sum')

In [139]:
# closed_type_2_lm = closed_pawn_type_2(date_list[1][0], date_list[1][1])

# closed_ty2_overview_lm = closed_type_2_lm.pivot_table(index='categoryname', columns='closed_type', values='pawnid',aggfunc='nunique')
# closed_ty2_overview_lm = closed_ty2_overview_lm.reset_index()
# closed_ty2_overview_amt_lm = closed_type_2_lm.pivot_table(index='categoryname', columns='closed_type', values='expected_amt',aggfunc='sum')
# closed_ty2_oc_lm = closed_type_2_lm.pivot_table(index=['categoryname', 'closed_type'], columns='oc_month_compare', values='pawnid', aggfunc='nunique')
# closed_ty2_oc_amt_lm = closed_type_2_lm.pivot_table(index=['categoryname', 'closed_type'], columns='oc_month_compare', values='expected_amt', aggfunc='sum')

In [140]:
# closed_ty2_overview_amt = closed_ty2_overview_amt.reset_index()

# closed_ty2_oc = closed_ty2_oc.reset_index()

# closed_ty2_oc_amt=closed_ty2_oc_amt.reset_index()

# closed_ty2_overview_amt_lm = closed_ty2_overview_amt_lm.reset_index()

# closed_ty2_oc_lm = closed_ty2_oc_lm.reset_index()

# closed_ty2_oc_amt_lm = closed_ty2_oc_amt_lm.reset_index()

In [141]:
client = pygsheets.authorize(service_account_file=r'D:\F88\Python code\Immediate_warning\service_account.json')

dailyrp_wb_id = '1WZp4LcjNVnGJ4Fn5KIC8Fg0L4MymwynRxgkLTnLTWAA'
# 1yapjrHmzWCEUr_CL94aUiNo7TLMKLPTyMefNiDZ8MmE
sheet = client.open_by_key(dailyrp_wb_id)

raw_sheet = sheet.worksheet_by_title('pivot_off_1')

raw_sheet_2 = sheet.worksheet_by_title('pivot_off_2')
raw_sheet_3 = sheet.worksheet_by_title('no_remind_list')

In [142]:
raw_sheet.set_dataframe(ontime_overview_1, (3,2))

In [143]:
raw_sheet.set_dataframe(ontime_overview_2, (15,2))

In [144]:
raw_sheet.set_dataframe(ontime_overview_1_last_month, (30,2))

In [145]:
raw_sheet.set_dataframe(ontime_overview_2_last_month, (45,2))

In [146]:
raw_sheet.set_dataframe(ontime_khuvuc_1, (60,2))

In [147]:
raw_sheet.set_dataframe(ontime_khuvuc_2, (75,2))

In [148]:
raw_sheet.set_dataframe(ontime_khuvuc_1_last_month, (90,2))

In [149]:
raw_sheet.set_dataframe(ontime_khuvuc_2_last_month, (105,2))

In [150]:
raw_sheet.set_dataframe(ontime_khuvuc_package_1, (120,2))

In [151]:
raw_sheet.set_dataframe(ontime_khuvuc_package_2, (140,2))

In [152]:
raw_sheet.set_dataframe(ontime_khuvuc_package_1_last_month, (170,2))

In [153]:
raw_sheet.set_dataframe(ontime_khuvuc_package_2_last_month, (190,2))

In [154]:
raw_sheet_2.set_dataframe(ontime_pgd_motor_title, (3,2))

In [155]:
raw_sheet_2.set_dataframe(ontime_pgd_motor_title_last_month, (503,2))

In [156]:
raw_sheet_2.set_dataframe(ontime_pgd_car_title, (1003,2))

In [157]:
raw_sheet_2.set_dataframe(ontime_pgd_car_title_last_month, (1503,2))

In [158]:
raw_sheet_2.set_dataframe(ontime_qlkv_motor_title, (2003,2))

In [159]:
raw_sheet_2.set_dataframe(ontime_qlkv_motor_title_last_month, (2103,2))

In [160]:
raw_sheet_2.set_dataframe(ontime_qlkv_car_title, (2203,2))

In [161]:
raw_sheet_2.set_dataframe(ontime_qlkv_car_title_last_month, (2303,2))

In [162]:
raw_sheet_2.set_dataframe(ontime_province_motor_title, (2403,2))

In [163]:
raw_sheet_2.set_dataframe(ontime_province_motor_title_last_month, (2503,2))

In [164]:
raw_sheet_2.set_dataframe(ontime_province_car_title, (2603,2))

In [165]:
raw_sheet_2.set_dataframe(ontime_province_car_title_last_month, (2703,2))

In [166]:
raw_sheet.set_dataframe(ontime_schedule, (210,2))

In [167]:
raw_sheet.set_dataframe(ontime_schedule_last_month, (260,2))

In [168]:
raw_sheet.set_dataframe(ontime_by_day_category, (310,2))

In [169]:
raw_sheet.set_dataframe(ontime_by_day_category_last_month, (360,2))

In [170]:
raw_sheet.set_dataframe(ontime_schedule_area_motortitle, (410,2))

In [171]:
raw_sheet.set_dataframe(ontime_schedule_area_motortitle_last_month, (450,2))

In [172]:
raw_sheet.set_dataframe(ontime_schedule_area_cartitle, (500,2))

In [173]:
raw_sheet.set_dataframe(ontime_schedule_area_cartitle_last_month, (550,2))

In [174]:
raw_sheet.set_dataframe(vn_ontime_schedule_area_motortitle_package, (600,2))

In [175]:
raw_sheet.set_dataframe(vt_ontime_schedule_area_motortitle_package, (650,2))

In [176]:
raw_sheet.set_dataframe(vn_ontime_schedule_area_motortitle_package_last_month, (700,2))

In [177]:
raw_sheet.set_dataframe(vt_ontime_schedule_area_motortitle_package_last_month, (750,2))

In [178]:
raw_sheet.set_dataframe(vn_ontime_schedule_area_cartitle_package, (800,2))

In [179]:
raw_sheet.set_dataframe(vt_ontime_schedule_area_cartitle_package, (850,2))

In [180]:
raw_sheet.set_dataframe(vn_ontime_schedule_area_cartitle_package_last_month, (900,2))

In [181]:
raw_sheet.set_dataframe(vt_ontime_schedule_area_cartitle_package_last_month, (950,2))

In [182]:
raw_sheet_2.set_dataframe(ontime_brr_ym_overview_motor_title, (2803,2))

In [183]:
raw_sheet_2.set_dataframe(ontime_brr_ym_overview_motor_title_last_month, (2853,2))

In [184]:
raw_sheet_2.set_dataframe(ontime_brr_ym_overview_car_title, (2903,2))

In [185]:
raw_sheet_2.set_dataframe(ontime_brr_ym_overview_car_title_last_month, (2953,2))

In [186]:
raw_sheet_2.set_dataframe(ontime_category_brr_ym_package_motor_title, (3003,2))

In [187]:
raw_sheet_2.set_dataframe(ontime_category_brr_ym_package_motor_title_last_month, (3053,2))

In [188]:
raw_sheet_2.set_dataframe(ontime_category_brr_ym_package_car_title, (3103,2))

In [189]:
raw_sheet_2.set_dataframe(ontime_category_brr_ym_package_car_title_last_month, (3153,2))

In [190]:
raw_sheet.set_dataframe(ontime_money_range_package, (1000,2))

In [191]:
raw_sheet.set_dataframe(ontime_money_range_package_last_month, (1030,2))

In [192]:
raw_sheet.set_dataframe(ontime_time_range_package, (1060,2))

In [193]:
raw_sheet.set_dataframe(ontime_time_range_package_last_month, (1090,2))

In [194]:
raw_sheet.set_dataframe(ontime_money_range_package_area, (1115,2))

In [195]:
raw_sheet.set_dataframe(ontime_money_range_package_area_last_month, (1140,2))

In [196]:
raw_sheet.set_dataframe(ontime_time_range_package_area,(1165,2))

In [197]:
raw_sheet.set_dataframe(ontime_time_range_package_area_last_month, (1190,2))

In [198]:
raw_sheet.set_dataframe(ontime_money_time_range, (1215,2))

In [199]:
raw_sheet.set_dataframe(ontime_money_time_range_last_month, (1240,2))

In [200]:
raw_sheet.set_dataframe(ontime_cus_type_overview, (1265,2))

In [201]:
raw_sheet.set_dataframe(ontime_cus_type_overview_last_month, (1270,2))

In [202]:
raw_sheet.set_dataframe(ontime_custype_category, (1280,2))

In [203]:
raw_sheet.set_dataframe(ontime_custype_category_last_month, (1290,2))

In [204]:
raw_sheet.set_dataframe(ontime_custype_category_package, (1300,2))

In [205]:
raw_sheet.set_dataframe(ontime_custype_category_package_last_month, (1315,2))

In [206]:
raw_sheet.set_dataframe(ontime_custype_category_area, (1330,2))

In [207]:
raw_sheet.set_dataframe(ontime_custype_category_area_last_month, (1340,2))

In [208]:
raw_sheet.set_dataframe(ontime_time_measurement, (1350,2))

In [209]:
raw_sheet.set_dataframe(ontime_time_measurement_last_month, (1370,2))

In [210]:
raw_sheet.set_dataframe(remind_motor_title_area, (1390,2))

In [211]:
raw_sheet.set_dataframe(remind_motor_title_area_last_month, (1400,2))

In [212]:
raw_sheet_2.set_dataframe(remind_motor_title_shop, (3203,2))

In [213]:
raw_sheet_2.set_dataframe(remind_motor_title_shop_last_month, (3703,2))

In [214]:
raw_sheet.set_dataframe(remind_motor_title_schedule, (1410,2))

In [215]:
raw_sheet.set_dataframe(remind_motor_title_schedule_last_month, (1460,2))

In [216]:
raw_sheet.set_dataframe(remind_car_title_area, (1510,2))

In [217]:
raw_sheet.set_dataframe(remind_car_title_area_last_month, (1520,2))

In [218]:
raw_sheet_2.set_dataframe(remind_car_title_shop, (4203,2))

In [219]:
raw_sheet_2.set_dataframe(remind_car_title_shop_last_month, (4703,2))

In [220]:
raw_sheet_2.set_dataframe(remind_motor_title_asm, (5203,2))

In [221]:
raw_sheet_2.set_dataframe(remind_motor_title_asm_last_month, (5303,2))

In [222]:
raw_sheet_2.set_dataframe(remind_car_title_asm, (5403,2))

In [223]:
raw_sheet_2.set_dataframe(remind_car_title_asm_last_month, (5503,2))

In [224]:
raw_sheet.set_dataframe(remind_car_title_schedule, (1530,2))

In [225]:
raw_sheet.set_dataframe(remind_car_title_schedule_last_month, (1580,2))

In [226]:
# raw_sheet.set_dataframe(admind_remind_summary, (1630,2))

In [227]:
# raw_sheet.set_dataframe(cmt_group_motor_title, (1680,2))

In [228]:
# raw_sheet.set_dataframe(cmt_group_car_title, (1685,2))

In [229]:
# raw_sheet.set_dataframe(closed_ty1_overview, (1690,2))

In [230]:
# raw_sheet.set_dataframe(closed_ty1_oc, (1700,2))

In [231]:
# raw_sheet.set_dataframe(closed_ty1_overview_lm, (1710,2))

In [232]:
# raw_sheet.set_dataframe(closed_ty1_oc_lm, (1720,2))

In [233]:
# raw_sheet.set_dataframe(closed_ty2_overview, (1730,2))

In [234]:
# raw_sheet.set_dataframe(closed_ty2_overview_amt, (1740,2))

In [235]:
# raw_sheet.set_dataframe(closed_ty2_oc, (1750,2))

In [236]:
# raw_sheet.set_dataframe(closed_ty2_oc_amt, (1760,2))

In [237]:
# raw_sheet.set_dataframe(closed_ty2_overview_lm, (1770,2))

In [238]:
# raw_sheet.set_dataframe(closed_ty2_overview_amt_lm, (1780,2))

In [239]:
# raw_sheet.set_dataframe(closed_ty2_oc_lm, (1790,2))

In [240]:
# raw_sheet.set_dataframe(closed_ty2_oc_amt_lm, (1810,2))

In [241]:
raw_sheet.set_dataframe(pmt_3days_by_day_cate, (1830,2))

In [242]:
raw_sheet.set_dataframe(pmt_3days_by_day_cate_last_month, (1880,2))

In [243]:
# raw_sheet_3.set_dataframe(admind_no_remind, (1,1))

In [244]:

raw_sheet.set_dataframe(ontime_schedule_3, (1915,1))

raw_sheet.set_dataframe(ontime_schedule_3_last_month, (1965,1))

In [245]:
raw_sheet.set_dataframe(ontime_schedule_4_vaynhanh, (2005,2))

In [246]:
raw_sheet.set_dataframe(ontime_schedule_4_vaythuong, (2040,2))

In [247]:
raw_sheet.set_dataframe(ontime_schedule_4_vaynhanh_last_month, (2075,2))

In [248]:
raw_sheet.set_dataframe(ontime_schedule_4_vaythuong_last_month, (2115,2))

In [249]:
raw_sheet_2.set_dataframe(remind_motor_title_province, (5600,2))

In [250]:
raw_sheet_2.set_dataframe(remind_motor_title_province_last_month, (5700,2))

In [251]:
raw_sheet_2.set_dataframe(remind_car_title_province, (5800,2))

In [252]:
raw_sheet_2.set_dataframe(remind_car_title_province_last_month, (5900,2))

In [253]:
shop_data.to_excel('shop_data.xlsx', index = False)

In [252]:
import xlsxwriter
writer = pd.ExcelWriter(r'ontime_raw_T12.xlsx', engine='xlsxwriter')
ontime_spdk.to_excel(writer, sheet_name='ontime_spdk')
ontime_spdk_last_month.to_excel(writer, sheet_name='ontime_spdk_last_month')
# shop_infor.to_excel(writer, sheet_name = 'shop_infor')
# full_tb_dkot.to_excel(writer, sheet_name = 'full_tb_dkot', index = False)
# full_tb_dkxm.to_excel(writer, sheet_name = 'full_tb_dkxm', index = False)
# ontime_spdk_last_month.to_excel(writer, sheet_name = 'ontime_spdk_last_month', index = False)
# balance_pre_days.to_excel(writer, sheet_name= 'balance_pre_days_summ', index = False)
writer.save()

In [ ]:
import xlsxwriter
writer3 = pd.ExcelWriter(r'ontime_dkxm_this_month_T12.xlsx', engine='xlsxwriter')
ontime_dkxm_this_month.to_excel(writer3, sheet_name='ontime_dkxm_this_month')

writer3.save()

In [86]:
remind_stats_motor_title.to_excel('remind_stats_motor_title_T12.xlsx', index = False)

In [87]:
remind_stats_car_title.to_excel('remind_stats_car_title_T12.xlsx', index = False)

In [242]:
remind_stats_car_title.head(5)

,pawnid,dpd_yesterday,packagecode,shopname,shopcode,areaid,scheduletodate,paydate,acc_prin_payment,status,...,Shop ID,Vùng/Miền,Thời gian KT,Vùng,code,name,GĐM,TPK,QLKV,khu_vuc
0,1264475,-1,None,F88 HCM - 128/1 Nguyễn Ảnh Thủ,SG2042,2.0,2020-12-04,2020-12-04,9872320.0,14,...,227,Miền Nam,2019 - 7,HCM,SG2042,F88 HCM - 128/1 Nguyễn Ảnh Thủ,Lê Ngọ,Trần Thanh Phú,Nguyễn Đình Nhật,TP. Hồ Chí Minh
1,1264475,-1,None,F88 HCM - 128/1 Nguyễn Ảnh Thủ,SG2042,2.0,2020-12-04,2020-12-04,9872320.0,14,...,227,Miền Nam,2019 - 7,HCM,SG2042,F88 HCM - 128/1 Nguyễn Ảnh Thủ,Lê Ngọ,Trần Thanh Phú,Nguyễn Đình Nhật,TP. Hồ Chí Minh
2,1264475,-1,None,F88 HCM - 128/1 Nguyễn Ảnh Thủ,SG2042,2.0,2020-12-04,2020-12-04,9872320.0,14,...,227,Miền Nam,2019 - 7,HCM,SG2042,F88 HCM - 128/1 Nguyễn Ảnh Thủ,Lê Ngọ,Trần Thanh Phú,Nguyễn Đình Nhật,TP. Hồ Chí Minh
3,1303996,-1,None,F88 HCM - 656 Lạc Long Quân,SG2003,2.0,2020-12-19,2020-12-19,119412050.0,14,...,183,Miền Nam,2018 - 11,HCM,SG2003,F88 HCM - 656 Lạc Long Quân,Lê Ngọ,Trần Thanh Phú,Lý Văn Đạt,TP. Hồ Chí Minh
4,1303996,-1,None,F88 HCM - 656 Lạc Long Quân,SG2003,2.0,2020-12-19,2020-12-19,119412050.0,14,...,183,Miền Nam,2018 - 11,HCM,SG2003,F88 HCM - 656 Lạc Long Quân,Lê Ngọ,Trần Thanh Phú,Lý Văn Đạt,TP. Hồ Chí Minh


In [240]:
remind_stats_motor_title.to_excel('remind_stats_motor_title_T12.xlsx', index = False)
# remind_stats_car_title.to_excel('remind_stats_car_title.xlsx', index = False)

In [241]:
remind_stats_motor_title[].to_excel('remind_stats_motor_title_T11.xlsx', index = False)
# remind_stats_car_title.to_excel('remind_stats_car_title.xlsx', index = False)

SyntaxError: invalid syntax (<ipython-input-241-d9715a50f001>, line 1)

In [ ]:
remind_stats_motor_title.columns

In [ ]:
remind_stats_motor_title['cmt_type'].unique()

In [ ]:
# Comment on Pos
# Comment group
cmt_group_motor_title= remind_stats_motor_title[(remind_stats_motor_title['commenttype'] != 'Admin') & (remind_stats_motor_title['cmt_type'] == '2.early_remind')].pivot_table(columns='commenttype', index = 'shopname', values='pawnid', aggfunc='nunique').reset_index()

cmt_group_car_title= remind_stats_car_title[(remind_stats_car_title['commenttype'] != 'Admin') & (remind_stats_motor_title['cmt_type'] == '2.early_remind')].pivot_table(columns='commenttype', index = 'shopname', values='pawnid', aggfunc='nunique').reset_index()


In [ ]:
cmt_group_motor_title = cmt_group_motor_title.fillna(0)

In [ ]:
cmt_group_car_title = cmt_group_car_title.fillna(0)

In [ ]:
writer2 = pd.ExcelWriter(r'shop_comment_group.xlsx', engine='xlsxwriter')
cmt_group_motor_title.to_excel(writer2, sheet_name='cmt_group_motor_title')
cmt_group_car_title.to_excel(writer2, sheet_name='cmt_group_car_title')
# shop_infor.to_excel(writer, sheet_name = 'shop_infor')
# full_tb_dkot.to_excel(writer, sheet_name = 'full_tb_dkot', index = False)
# full_tb_dkxm.to_excel(writer, sheet_name = 'full_tb_dkxm', index = False)
# ontime_spdk_last_month.to_excel(writer, sheet_name = 'ontime_spdk_last_month', index = False)
# balance_pre_days.to_excel(writer, sheet_name= 'balance_pre_days_summ', index = False)
writer2.save()

In [ ]:
cmt_group_car_title

In [ ]:
# Early remind rate trending
check_date = '2020-11-12'